In [1]:
import pandas as pd
import geopandas as gpd
from IPython.display import display
import requests as re
from io import BytesIO
from os import path, makedirs


from core.downloads.geosampa import get_capabilities, get_features

# GT Saúde

O Grupo de Trabalho de Saúde definiu os indicadores sem a utilização dos formulários, em reuniões posteriores. Por isso, os indicadores serão descritos diretamente neste notebook para documentação.

A base inicial para os indicadores é o capítulo 7, Eficiência nas macrorregiões de saúde no Sistema Único de Saúde: uma abordagem comparativa – 2008-2017, do livro [SUS: avaliação da eficiência do gasto público em saúde](https://repositorio.ipea.gov.br/handle/11058/12029). Este capítulo busca associar os custos e infraestrutura relacionada ao SUS e o impacto na saúde da população de cada macrorregião, utilizando as seguintes variáveis:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Despesas per capita em atenção básica e vigilância;
- Despesas per capita em atenção especializada e Suporte profilático terapêutico;
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Equipamentos respiradores e ventiladores SUS/hab. (x100.000);
- População (em mil hab.);
- Área (km2);
- Densidade demográfica (hab./km²);
- Idosos %;
- PIB per capita;
- Usuários de planos de saúde %.


A partir dessas referência, o GT avaliou a disponibilidade de dados semelhantes a nível intramunicipal e complementou com informações de produção dos serviços de saúde, chegando na seguinte lista de variáveis, sempre que possível agregando por Subprefeitura:

- Taxa de mortalidade por causas evitáveis (100.000 hab.);
- Taxa de mortalidade por causas tratáveis (100.000 hab.);
- Taxa de mortalidade por causas evitáveis/tratáveis (100.000 hab.);
- Leitos de internações/hab. (x100.000);
- Leitos complementares/hab. (x100.000);
- Leitos de UTI/hab. (x100.000);
- Despesas liquidadas em atenção básica e vigilância;
- Despesas liquidadas em atenção especializada e Suporte profilático terapêutico;
- Consultas na Atenção Básica;
- Consultas na Atenção Especializada;
- Consultas na Urgência/Emergência;
- Número de Unidades Básicas de Saúde;
- Número de Ambulatórios especializados;
- Número de Unidades de Saúde Mental;
- Número de Unidades de Urgência e Emergência;
- Número de Hospitais;
- Número de Unidades DST/AIDS;
- Número de demais unidades;

A partir dessa definição, vamos obter as variáveis.

# Extração das variáveis e transformação inicial

## Número de unidades

Estabelecimentos de saúde municipais, estaduais, federais e privados do município estão disponíveis no GeoSampa, em diferentes camadas. Primeiro, vamos avaliar quais são essas camadas e, depois, baixá-las e agrupá-las da maneira decidida pelo GT.

In [2]:
equipamento_saude_typenames = get_capabilities('equipamento_saude')
equipamento_saude_typenames

[{'name': 'geoportal:equipamento_saude_abrangencia_ubs',
  'title': 'Abrangência das Unidades Básicas de Saúde',
  'abstract': 'Áreas de abrangência das UBS.'},
 {'name': 'geoportal:equipamento_saude_ambulatorios_especializados',
  'title': 'Ambulatórios especializados',
  'abstract': 'Ambulatórios Especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Atende pacientes encaminhados por outros estabelecimentos e que requerem cuidados em áreas específicas.'},
 {'name': 'geoportal:equipamento_saude_coordenadoria_regional',
  'title': 'Coordenadoria Regional de Saúde',
  'abstract': 'Limites administrativos da Coordenadoria Regional de Saúde.'},
 {'name': 'geoportal:equipamento_saude_hospital',
  'title': 'Hospital',
  'abstract': 'Hospitais do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Estabelecimentos de saúde com capacidade de internamento (leitos) e que real

Primeiro, removemos todas as camadas que não representam os pontos das unidades de saúde. Além disso, a camada de vigilância não retorna nenhuma unidade, então vamos removê-la também.

In [3]:
del equipamento_saude_typenames[0]
del equipamento_saude_typenames[1]
del equipamento_saude_typenames[4]
del equipamento_saude_typenames[8]
del equipamento_saude_typenames[7]

equipamento_saude_typenames

[{'name': 'geoportal:equipamento_saude_ambulatorios_especializados',
  'title': 'Ambulatórios especializados',
  'abstract': 'Ambulatórios Especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Atende pacientes encaminhados por outros estabelecimentos e que requerem cuidados em áreas específicas.'},
 {'name': 'geoportal:equipamento_saude_hospital',
  'title': 'Hospital',
  'abstract': 'Hospitais do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo. Estabelecimentos de saúde com capacidade de internamento (leitos) e que realizam procedimentos de média e alta complexidade.'},
 {'name': 'geoportal:equipamento_saude_outros',
  'title': 'Saúde - Outros',
  'abstract': 'Estabelecimentos e serviços especializados do cadastro de estabelecimentos de saúde municipais, estaduais, federais e privados da Cidade de São Paulo.'},
 {'name': 'geoportal:equipamento_saude_saude_mental

In [4]:
gdfs_equip = []

for t in equipamento_saude_typenames:
    gdfs_equip.append(get_features(t.get('name')))

In [5]:
for gdf in gdfs_equip:
    display(gdf.crs)
    display(gdf.head(2))

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ambulatorios_especializados....,120001,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,963,AMA 12H BORACEA - DR. LUIZ BACCALA,"R. DR. RIBEIRO DE ALMEIDA, 14",BARRA FUNDA,01137-020,None,4210-5396 / 3392-1944,None,POINT (331397.001 7397422.322)
1,equipamento_saude_ambulatorios_especializados....,120002,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,935,AMA 12H COMPLEXO PRATES,"R. PRATES, 1101",BOM RETIRO,01121-000,None,3229-8192,None,POINT (333135.515 7397559.623)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_hospital.120093,120093,6,SAUDE,0,SEM REDE,Sem descrição,32,Hospital Especializado,É o estabelecimento de saúde destinado a prest...,...,Administração estadual vinculada à Secretaria ...,959,AMB ESPEC DR. GERALDO PAULO BOURROL (CONSOLAÇÃO),"R. MARTINS FONTES, 208",CENTRO,01050-000,None,2141-6200,None,POINT (332182.763 7394765.185)
1,equipamento_saude_hospital.120108,120108,6,SAUDE,0,SEM REDE,Sem descrição,32,Hospital Especializado,É o estabelecimento de saúde destinado a prest...,...,Administração estadual vinculada à Secretaria ...,366,AME DR. LUIZ ROBERTO BARRADAS BARATA,"AV. ALMIRANTE DELAMARE, 1534",CIDADE NOVA HELIOPOLIS,04230-000,None,2065-1550,None,POINT (337785.173 7388015.086)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

...

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ubs_posto_centro.120031,120031,6,SAUDE,0,SEM REDE,Sem descrição,45,Unidade Básica de Saúde,"Atendimento básico em Pediatria, Ginecologia, ...",...,Administração municipal,580,AMA/UBS ÁGUA RASA - DR. MARCOS ANDRADE CORSATO,"R. SERRA DE JAIRE, 1480",ÁGUA RASA,03175-001,None,2605-5307,None,POINT (338714.702 7394301.385)
1,equipamento_saude_ubs_posto_centro.120032,120032,6,SAUDE,0,SEM REDE,Sem descrição,45,Unidade Básica de Saúde,"Atendimento básico em Pediatria, Ginecologia, ...",...,Administração municipal,429,AMA/UBS AGUIA DE HAIA,"R. TANTAS PALAVRAS, 59",COHAB ÁGUIA DE HAIA,08223-510,None,3756-3264,None,POINT (350002.697 7397919.976)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_unidades_dst-aids.120363,120363,6,SAUDE,0,SEM REDE,Sem descrição,19,Centro de Testagem e Acompanhamento DST/AIDS,Serviço estruturado para atividades de prevenç...,...,Administração municipal,224,CTA IST/AIDS CIDADE TIRADENTES,"R. MILAGRE DOS PEIXES, 357",CONJUNTO HABITACIONA,08474-120,None,2282-7055 / 2964-0784,None,POINT (356240.559 7391275.466)
1,equipamento_saude_unidades_dst-aids.120364,120364,6,SAUDE,0,SEM REDE,Sem descrição,19,Centro de Testagem e Acompanhamento DST/AIDS,Serviço estruturado para atividades de prevenç...,...,Administração municipal,319,CTA IST/AIDS GUAIANASES,"R. CENTRALINA, 168",GUAIANASES,08410-000,None,2554-5312,None,POINT (355449.169 7395443.615)


<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_urgencia_emergencia.120007,120007,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,78,AMA 24H CAPÃO REDONDO,"AV. COMENDADOR SANTANA, 774",JD BOA ESPERANÇA,05866-000,None,5872-9917 / 5872-9919,None,POINT (319131.694 7380616.296)
1,equipamento_saude_urgencia_emergencia.120008,120008,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,646,AMA 24H ENGENHEIRO GOULART - DR. JOSÉ PIRES,"R. AUGUSTO CORREIA LEITE, 538",ENG. GOULART,03726-100,None,2038-1626 / 2622-0366,None,POINT (345081.544 7400275.725)


In [6]:
gdf_equip = pd.concat(gdfs_equip)
gdf_equip = gpd.GeoDataFrame(gdf_equip)
gdf_equip = gdf_equip.set_geometry(gdfs_equip[0].geometry.name)
gdf_equip = gdf_equip.set_crs(gdfs_equip[0].crs)
gdf_equip

,id,cd_identificador,cd_identificador_tema_equipamento,nm_tema_equipamento,cd_identificador_rede_equipamento,nm_rede_equipamento,tx_rede_equipamento,cd_identificador_tipo_equipamento,nm_tipo_equipamento,tx_tipo_equipamento,...,tx_esfera_administrativa_equipamento,cd_equipamento,nm_equipamento,tx_endereco_equipamento,nm_bairro_equipamento,cd_cep_equipamento,tx_horario_funcionamento,tx_numero_telefone,cd_equipamento_base_origem,geometry
0,equipamento_saude_ambulatorios_especializados....,120001,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,963,AMA 12H BORACEA - DR. LUIZ BACCALA,"R. DR. RIBEIRO DE ALMEIDA, 14",BARRA FUNDA,01137-020,None,4210-5396 / 3392-1944,None,POINT (331397.001 7397422.322)
1,equipamento_saude_ambulatorios_especializados....,120002,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,935,AMA 12H COMPLEXO PRATES,"R. PRATES, 1101",BOM RETIRO,01121-000,None,3229-8192,None,POINT (333135.515 7397559.623)
2,equipamento_saude_ambulatorios_especializados....,120003,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,17,AMA 12H JARDIM PERI PERI,"R. JOÃO GUERRA, 247",JD PERI PERI,05535-100,None,3744-3021 / 3744-3082,None,POINT (323248.398 7390472.108)
3,equipamento_saude_ambulatorios_especializados....,120004,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,546,AMA 12H JARDIM SÃO LUIZ,"R. LUIZ ANTONIO VERNEY, 98",JD SÃO LUIZ,05846-380,None,5511-1153 / 5816-5539,None,POINT (322162.36 7382246.957)
4,equipamento_saude_ambulatorios_especializados....,120005,6,SAUDE,0,SEM REDE,Sem descrição,4,Assistência Médica Ambulatorial,"Atua na área de atenção básica, integrada e ar...",...,Administração municipal,658,AMA 12H MAURICE PATE,"R. FREI GERMANO, 50",PENHA,03604-050,None,2097-5802,None,POINT (341684.865 7397317.38)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,equipamento_saude_urgencia_emergencia.121025,121025,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,543,UPA VERA CRUZ,"AV. DOS FUNCIONARIOS PUBLICOS, 379",JARDIM VERA CRUZ,04962-000,None,5898-5900,None,POINT (318482.017 7373748.846)
49,equipamento_saude_urgencia_emergencia.121026,121026,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,952,UPA VERGUEIRO,"R. VERGUEIRO, 613",LIBERDADE,01504-001,None,4040-1067,None,POINT (332661.412 7392701.246)
50,equipamento_saude_urgencia_emergencia.121027,121027,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1008,UPA VILA MARIA - DR JOSE MAURO DEL ROIO CORREA,"PRAÇA ENGENHEIRO HUGO BRANDI, 15",VILA MARIA BAIXA,02168-050,None,3500-2560,None,POINT (338962.421 7398328.416)
51,equipamento_saude_urgencia_emergencia.121028,121028,6,SAUDE,0,SEM REDE,Sem descrição,38,Pronto Atendimento,Atendimento de urgência dentro do horário de f...,...,Administração municipal,1034,UPA VILA MARIANA,"R. DIOGO DE FARIA, 609",V CLEMENTINO,04037-002,None,3245-9010,None,POINT (332386.864 7389557.568)


## Consultas médicas

Os dados de consultas médicas serão obtidos do boletim "Saúde em dados", disponibilizado anualmente pela CEINFO, da Secretaria Municipal de Saúde. Existe uma versão do boletim em excel, regionalizado por subprefeituras. Primeiro, vamos confirmar as planilhas disponíveis no arquivo.

In [7]:
url = 'https://capital.sp.gov.br/documents/d/saude/tabelas_ceinfo_dados_sub_2024_v3_rev09012025'

response = re.get(url)
response.raise_for_status()
xlsx_file = BytesIO(response.content)
excel = pd.ExcelFile(xlsx_file)

excel.sheet_names

['População',
 'Nascidos Vivos',
 'Grupos de Robson',
 'Mortalidade',
 'Neoplasia',
 'Doenças Notificação Compulsória',
 'Agravos',
 'COVID-19',
 'Estado Nutricional',
 'Cobertura Assist SIGA-Saúde',
 'Consultas Medicas_Odontológicas',
 'Consultas Médicas Especializada',
 'Cons Prof Nivel Superior SMS',
 'Cons Prof Nivel Superior SES',
 'Assistência Hospitalar',
 'Apoio Diagnóstico',
 'Rede Física',
 'Leitos']

A tabela desejada é a `Consultas Medicas_Odontológicas`, então vamos abri-la como um dataframe.

In [8]:
sheet_name = 'Consultas Medicas_Odontológicas'
header = [0, 1]
skiprows = 5
df_consultas = pd.read_excel(xlsx_file,
                   sheet_name=sheet_name,
                   skiprows=skiprows,
                   header=header,
                   thousands='.')
df_consultas

Unnamed: 0_level_0                                      Subprefeitura  \
   Unnamed: 0_level_1                                 Unnamed: 1_level_1   
0                 NaN                          Aricanduva/Formosa/Carrão   
1                 NaN                                            Butantã   
2                 NaN                                        Campo Limpo   
3                 NaN                                  Capela do Socorro   
4                 NaN                            Casa Verde/Cachoeirinha   
5                 NaN                                      Cidade Ademar   
6                 NaN                                  Cidade Tiradentes   
7                 NaN                                 Ermelino Matarazzo   
8                 NaN                              Freguesia/Brasilândia   
9                 NaN                                         Guaianases   
10                NaN                                           Ipiranga   
11                NaN                                     Itaim Paulista   
12                NaN                                           Itaquera   
13                NaN                                          Jabaquara   
14                NaN                                    Jaçanã/Tremembé   
15                NaN                                               Lapa   
16                NaN                                        M'Boi Mirim   
17                NaN                                              Mooca   
18                NaN                                        Parelheiros   
19                NaN                                              Penha   
20                NaN                                              Perus   
21                NaN                                          Pinheiros   
22                NaN                                   Pirituba/Jaraguá   
23                NaN                                   Santana/Tucuruvi   
24                NaN                                        Santo Amaro   
25                NaN                                         São Mateus   
26                NaN                                São Miguel Paulista   
27                NaN                                          Sapopemba   
28                NaN                                                 Sé   
29                NaN                          Vila Maria/Vila Guilherme   
30                NaN                                       Vila Mariana   
31                NaN                                      Vila Prudente   
32                NaN                             Município de São Paulo   
33                NaN                                                NaN   
34                NaN  Fonte: Sistema de Informação Ambulatorial - SI...   
35                NaN                                                NaN   
36                NaN  Nota: (1) A Rede SUS é composta por estabeleci...   
37                NaN                                                NaN   
38                NaN                                                NaN   
39                NaN                                                NaN   
40                NaN                                                NaN   
41                NaN                                                NaN   

   Consulta Médica\nna Atenção Básica(2)                            \
                       Não Urgência\nSMS Urgência\nSMS       Total   
0                               188894.0       93846.0    282740.0   
1                               443917.0      212283.0    656200.0   
2                               837251.0      652006.0   1489257.0   
3                               554063.0      283894.0    837957.0   
4                               327363.0      258095.0    585458.0   
5                               645314.0      188878.0    834192.0   
6                               325739.0       34662.0    360401.0   
7                           

Como a tabela original vem em um formato amigável para humanos, precisamos ajustá-la para melhorar a sua utilização computacional. A primeira coisa é filtrar linhas e colunas carregadas por engano, por conter informações de fonte ou caracteres não imprimíveis.

In [9]:
df_consultas = df_consultas.iloc[:32, 1:]

df_consultas

Subprefeitura Consulta Médica\nna Atenção Básica(2)  \
           Unnamed: 1_level_1                     Não Urgência\nSMS   
0   Aricanduva/Formosa/Carrão                              188894.0   
1                     Butantã                              443917.0   
2                 Campo Limpo                              837251.0   
3           Capela do Socorro                              554063.0   
4     Casa Verde/Cachoeirinha                              327363.0   
5               Cidade Ademar                              645314.0   
6           Cidade Tiradentes                              325739.0   
7          Ermelino Matarazzo                              258367.0   
8       Freguesia/Brasilândia                              476514.0   
9                  Guaianases                              358419.0   
10                   Ipiranga                              436257.0   
11             Itaim Paulista                              506647.0   
12                   Itaquera                              616893.0   
13                  Jabaquara                              202175.0   
14            Jaçanã/Tremembé                              333609.0   
15                       Lapa                              185920.0   
16                M'Boi Mirim                              927985.0   
17                      Mooca                              237620.0   
18                Parelheiros                              165641.0   
19                      Penha                              395582.0   
20                      Perus                              174019.0   
21                  Pinheiros                               98130.0   
22           Pirituba/Jaraguá                              499630.0   
23           Santana/Tucuruvi                              132926.0   
24                Santo Amaro                              115999.0   
25                 São Mateus                              578502.0   
26        São Miguel Paulista                              525496.0   
27                  Sapopemba                              388165.0   
28                         Sé                              308922.0   
29  Vila Maria/Vila Guilherme                              312691.0   
30               Vila Mariana                               90845.0   
31              Vila Prudente                              257419.0   

                            Consulta Médica\nna Atenção Especializada  \
   Urgência\nSMS      Total                              Gestão \nSMS   
0        93846.0   282740.0                                     56651   
1       212283.0   656200.0                                     75444   
2       652006.0  1489257.0                                    217035   
3       283894.0   837957.0                                    184580   
4       258095.0   585458.0                                    155265   
5       188878.0   834192.0                                    130523   
6        34662.0   360401.0                                     37112   
7       162909.0   421276.0                                     81975   
8       120546.0   597060.0                                    167701   
9        50300.0   408719.0                                     42217   
10      253211.0   689468.0                                    235574   
11        5370.0   512017.0                                     60996   
12      166986.0   783879.0                                    166236   
13      106137.0   308312.0                                     83267   
14       68684.0   402293.0                                     38275   
15      191187.0   377107.0                                    132722   
16      517440.0  1445425.0                                    248403   
17      118979.0   356599.0                                    487398   
18       67455.0   233096.0                                     23280   
19      564046.0   959628.0                                    13358

Depois, precisamos remover as colunas de totais, que resultariam em duplicação dos valores caso fossem mantidas.

In [10]:
cols_total_level0 = df_consultas.columns.get_level_values(0).str.contains('total', case=False)
cols_total_level1 = df_consultas.columns.get_level_values(1).str.contains('total', case=False)

df_consultas = df_consultas.loc[:, ~cols_total_level0 & ~cols_total_level1]

df_consultas

Subprefeitura Consulta Médica\nna Atenção Básica(2)  \
           Unnamed: 1_level_1                     Não Urgência\nSMS   
0   Aricanduva/Formosa/Carrão                              188894.0   
1                     Butantã                              443917.0   
2                 Campo Limpo                              837251.0   
3           Capela do Socorro                              554063.0   
4     Casa Verde/Cachoeirinha                              327363.0   
5               Cidade Ademar                              645314.0   
6           Cidade Tiradentes                              325739.0   
7          Ermelino Matarazzo                              258367.0   
8       Freguesia/Brasilândia                              476514.0   
9                  Guaianases                              358419.0   
10                   Ipiranga                              436257.0   
11             Itaim Paulista                              506647.0   
12                   Itaquera                              616893.0   
13                  Jabaquara                              202175.0   
14            Jaçanã/Tremembé                              333609.0   
15                       Lapa                              185920.0   
16                M'Boi Mirim                              927985.0   
17                      Mooca                              237620.0   
18                Parelheiros                              165641.0   
19                      Penha                              395582.0   
20                      Perus                              174019.0   
21                  Pinheiros                               98130.0   
22           Pirituba/Jaraguá                              499630.0   
23           Santana/Tucuruvi                              132926.0   
24                Santo Amaro                              115999.0   
25                 São Mateus                              578502.0   
26        São Miguel Paulista                              525496.0   
27                  Sapopemba                              388165.0   
28                         Sé                              308922.0   
29  Vila Maria/Vila Guilherme                              312691.0   
30               Vila Mariana                               90845.0   
31              Vila Prudente                              257419.0   

                 Consulta Médica\nna Atenção Especializada               \
   Urgência\nSMS                              Gestão \nSMS Gestão \nSES   
0        93846.0                                     56651            -   
1       212283.0                                     75444        98066   
2       652006.0                                    217035        10527   
3       283894.0                                    184580       108804   
4       258095.0                                    155265        61589   
5       188878.0                                    130523            -   
6        34662.0                                     37112            -   
7       162909.0                                     81975            -   
8       120546.0                                    167701        23488   
9        50300.0                                     42217         3236   
10      253211.0                                    235574       254921   
11        5370.0                                     60996        27276   
12      166986.0                                    166236       214906   
13      106137.0                                     83267           87   
14       68684.0                                     38275            -   
15      191187.0                                    132722        35962   
16      517440.0                                    248403            -   
17      118979.0                                    487398       125884   
18       67455.0                                     23280            -   
19      564046.0          

Agora, vamos transformar os níveis das colunas em novas colunas de valores (despivotar a tabela).

In [11]:
id_col = df_consultas.columns[0]
df_consultas_ajustado = pd.melt(df_consultas, id_vars=[id_col])
df_consultas_ajustado

,"(Subprefeitura, Unnamed: 1_level_1)",variable_0,variable_1,value
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
219,Sapopemba,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,9873.0
220,Sé,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,27847.0
221,Vila Maria/Vila Guilherme,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,16263.0
222,Vila Mariana,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,3015.0


Agora, renomeamos as colunas.

In [12]:
df_consultas_ajustado.columns = ['Subprefeitura', 'Categoria', 'Subcategoria', 'Qtd_Consultas']
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
219,Sapopemba,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,9873.0
220,Sé,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,27847.0
221,Vila Maria/Vila Guilherme,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,16263.0
222,Vila Mariana,Primeira Consulta Odontológica \nSMS,Unnamed: 14_level_1,3015.0


Em seguida, removemos as linhas de consultas odontológicas.

In [13]:
df_consultas_ajustado = df_consultas_ajustado[~df_consultas_ajustado['Categoria'].str.contains('Odontológica')]
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894.0
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917.0
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251.0
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063.0
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363.0
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,-
190,Vila Mariana,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,139951


Ainda, substituímos os `-` por 0 na coluna de quantidade de consultas.

In [14]:
hifen = df_consultas_ajustado['Qtd_Consultas'] == '-'
df_consultas_ajustado.loc[hifen, 'Qtd_Consultas'] = 0
df_consultas_ajustado.loc[:, 'Qtd_Consultas'] = df_consultas_ajustado.loc[:, 'Qtd_Consultas'].astype(int)
df_consultas_ajustado

/tmp/ipykernel_5635/3378969537.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consultas_ajustado.loc[hifen, 'Qtd_Consultas'] = 0


,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,188894
1,Butantã,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica\nna Atenção Básica(2),Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento\nem Urgência/Emerg...,Gestão \nSES,139951


Por último, vamos corrigir os nomes das categorias.

In [15]:
df_consultas_ajustado['Categoria'].value_counts()

Categoria
Consulta Médica\nna Atenção Básica(2)                     64
Consulta Médica\nna Atenção Especializada                 64
Consulta Médica/Atendimento\nem Urgência/Emergência(3)    64
Name: count, dtype: int64

In [16]:
df_consultas_ajustado.loc[:, 'Categoria'] = df_consultas_ajustado.loc[:, 'Categoria'].apply(lambda c: 'Consulta Médica na Atenção Básica' if 'Atenção Básica' in c else c)

df_consultas_ajustado.loc[:, 'Categoria'] = df_consultas_ajustado.loc[:, 'Categoria'].apply(lambda c: 'Consulta Médica/Atendimento em Urgência/Emergência' if 'Urgência' in c else c)

df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951


## Leitos de internação e complementares

Os dados de leitos não estão disponíveis de maneira regionalizada no boletim "Saúde em dados", então terão de ser obtidos de outra fonte. Eles estão disponíveis na plataforma [TabNet](http://tabnet.saude.prefeitura.sp.gov.br/cgi/deftohtm3.exe?secretarias/saude/TABNET/cnes/leito.def). Entretanto, os links dos arquivos gerados nessa plataforma podem não ser permanentes, então utilizaremos uma versão já baixada do arquivo csv.

Obs: caso a necessidade fosse mais geral ou a atualização mais frequente, seria possível desenvolver um *scrapper* para a plataforma, mas estrapola o escopo deste trabalho.

In [17]:
data_path = path.join('dados', 'saude')
leitos_filename = 'A161721192_29_142_217.csv'
leitos_file_path = path.join(data_path, leitos_filename)

df_leito = pd.read_csv(leitos_file_path,
                        skiprows = 3,
                        sep = ';',
                        encoding = 'latin1')
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA,Total
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-,95.0
1,Butantã,78,131,66,57,70,8,30,440.0
2,Campo Limpo,-,-,-,-,-,88,7,95.0
3,Capela do Socorro,78,155,96,62,38,28,-,457.0
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4,387.0
5,Cidade Ademar,-,-,-,-,-,-,14,14.0
6,Cidade Tiradentes,49,54,45,38,27,10,1,224.0
7,Ermelino Matarazzo,85,58,44,40,29,-,-,256.0
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13,387.0
9,Guaianases,40,58,46,48,33,24,-,249.0


Primeiro, vamos excluir as linhas sobressalentes ao final do dataframe e a coluna de totalização.

In [18]:
df_leito = df_leito.iloc[:32]
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA,Total
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-,95.0
1,Butantã,78,131,66,57,70,8,30,440.0
2,Campo Limpo,-,-,-,-,-,88,7,95.0
3,Capela do Socorro,78,155,96,62,38,28,-,457.0
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4,387.0
5,Cidade Ademar,-,-,-,-,-,-,14,14.0
6,Cidade Tiradentes,49,54,45,38,27,10,1,224.0
7,Ermelino Matarazzo,85,58,44,40,29,-,-,256.0
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13,387.0
9,Guaianases,40,58,46,48,33,24,-,249.0


In [19]:
cols_total = df_leito.columns.str.lower().str.contains('total')
df_leito = df_leito.loc[:,~cols_total]
df_leito

,Pref Regional,1-Cirúrgico,2-Clínico,3-Complementar,4-Obstétrico,5-Pediátrico,6-Outras Especialidades,7-Hospital/DIA
0,Aricanduva/ Formosa/ Carrão,-,-,-,-,-,95,-
1,Butantã,78,131,66,57,70,8,30
2,Campo Limpo,-,-,-,-,-,88,7
3,Capela do Socorro,78,155,96,62,38,28,-
4,Casa Verde/ Cachoeirinha,86,74,104,86,33,-,4
5,Cidade Ademar,-,-,-,-,-,-,14
6,Cidade Tiradentes,49,54,45,38,27,10,1
7,Ermelino Matarazzo,85,58,44,40,29,-,-
8,Freguesia/ Brasilândia,107,138,85,-,28,16,13
9,Guaianases,40,58,46,48,33,24,-


Em seguida, "despivotamos" a tabela.

In [20]:
df_leito_ajustado = df_leito.melt(id_vars=['Pref Regional'],
                                    var_name='Categoria específica',
                                    value_name='Qtd leitos')
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,-
1,Butantã,1-Cirúrgico,78
2,Campo Limpo,1-Cirúrgico,-
3,Capela do Socorro,1-Cirúrgico,78
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86
...,...,...,...
219,Sapopemba,7-Hospital/DIA,6
220,Sé,7-Hospital/DIA,69
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6
222,Vila Mariana,7-Hospital/DIA,166


Por último, substituimos os `-` por 0.

In [21]:
hifen = df_leito_ajustado['Qtd leitos'].str.contains('-')
df_leito_ajustado.loc[hifen,'Qtd leitos'] = 0
df_leito_ajustado.loc[:,'Qtd leitos'] = df_leito_ajustado.loc[:,'Qtd leitos'].astype(int)
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0
1,Butantã,1-Cirúrgico,78
2,Campo Limpo,1-Cirúrgico,0
3,Capela do Socorro,1-Cirúrgico,78
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86
...,...,...,...
219,Sapopemba,7-Hospital/DIA,6
220,Sé,7-Hospital/DIA,69
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6
222,Vila Mariana,7-Hospital/DIA,166


Posteriormente, cada categoria específica dos leitos será categorizada em Leitos de internações, leitos complementares ou leitos de UTI, de acordo com as variáveis definidas.

## Mortalidade

Os dados sobre mortalidade também serão extraídos da plataforma [TabNet](http://tabnet.saude.prefeitura.sp.gov.br/cgi/deftohtm3.exe?secretarias/saude/TABNET/cnes/leito.def). Novamente, como os links dos arquivos gerados nessa plataforma podem não ser permanentes, utilizaremos uma versão já baixada do arquivo csv.

In [22]:
data_path = path.join('dados', 'saude')
mortalidade_filename = 'A171717192_29_141_228.csv'
mortalidade_file_path = path.join(data_path, mortalidade_filename)

df_mortalidade = pd.read_csv(mortalidade_file_path,
                        skiprows = 4,
                        sep = ';',
                        encoding = 'latin1')
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,...,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte,Total
0,Aricanduva/Formosa/Carrão,4,9.0,-,9.0,6.0,2,3,3,15.0,...,38.0,10.0,5,19.0,3,3,46.0,1,260.0,1177.0
1,Butantã,5,10.0,1,12.0,12.0,3,2,8,20.0,...,62.0,13.0,10,23.0,6,10,66.0,3,292.0,1380.0
2,Campo Limpo,7,16.0,-,17.0,23.0,7,7,19,34.0,...,119.0,30.0,15,22.0,9,18,104.0,8,459.0,2254.0
3,Capela do Socorro,7,16.0,-,24.0,15.0,3,3,20,35.0,...,99.0,44.0,25,37.0,15,35,80.0,6,509.0,2464.0
4,Casa Verde/Cachoeirinha,4,15.0,-,27.0,19.0,1,3,5,19.0,...,75.0,14.0,5,11.0,5,5,52.0,3,288.0,1438.0
5,Cidade Ademar,-,20.0,2,13.0,14.0,3,1,14,26.0,...,78.0,12.0,8,18.0,14,9,75.0,3,312.0,1650.0
6,Cidade Tiradentes,1,12.0,1,14.0,9.0,1,2,4,7.0,...,45.0,2.0,1,14.0,3,6,51.0,1,183.0,921.0
7,Ermelino Matarazzo,3,10.0,-,12.0,7.0,-,1,5,14.0,...,30.0,4.0,2,16.0,6,3,36.0,-,200.0,906.0
8,Freguesia/Brasilândia,6,20.0,-,23.0,16.0,2,3,11,25.0,...,97.0,19.0,13,26.0,5,8,80.0,2,353.0,1872.0
9,Guaianases,1,16.0,-,16.0,12.0,1,1,4,13.0,...,55.0,2.0,4,10.0,-,5,68.0,3,255.0,1217.0


Primeiro, vamos excluir as linhas sobressalentes ao final do dataframe e a coluna de totalização.

In [23]:
df_mortalidade = df_mortalidade.iloc[:32]
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,...,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte,Total
0,Aricanduva/Formosa/Carrão,4,9.0,-,9.0,6.0,2,3,3,15.0,...,38.0,10.0,5,19.0,3,3,46.0,1,260.0,1177.0
1,Butantã,5,10.0,1,12.0,12.0,3,2,8,20.0,...,62.0,13.0,10,23.0,6,10,66.0,3,292.0,1380.0
2,Campo Limpo,7,16.0,-,17.0,23.0,7,7,19,34.0,...,119.0,30.0,15,22.0,9,18,104.0,8,459.0,2254.0
3,Capela do Socorro,7,16.0,-,24.0,15.0,3,3,20,35.0,...,99.0,44.0,25,37.0,15,35,80.0,6,509.0,2464.0
4,Casa Verde/Cachoeirinha,4,15.0,-,27.0,19.0,1,3,5,19.0,...,75.0,14.0,5,11.0,5,5,52.0,3,288.0,1438.0
5,Cidade Ademar,-,20.0,2,13.0,14.0,3,1,14,26.0,...,78.0,12.0,8,18.0,14,9,75.0,3,312.0,1650.0
6,Cidade Tiradentes,1,12.0,1,14.0,9.0,1,2,4,7.0,...,45.0,2.0,1,14.0,3,6,51.0,1,183.0,921.0
7,Ermelino Matarazzo,3,10.0,-,12.0,7.0,-,1,5,14.0,...,30.0,4.0,2,16.0,6,3,36.0,-,200.0,906.0
8,Freguesia/Brasilândia,6,20.0,-,23.0,16.0,2,3,11,25.0,...,97.0,19.0,13,26.0,5,8,80.0,2,353.0,1872.0
9,Guaianases,1,16.0,-,16.0,12.0,1,1,4,13.0,...,55.0,2.0,4,10.0,-,5,68.0,3,255.0,1217.0


In [24]:
cols_total = df_mortalidade.columns.str.lower().str.contains('total')
df_mortalidade = df_mortalidade.loc[:,~cols_total]
df_mortalidade

,Subprefeitura residência,Doencas infec intest,Tuberculose,Infec menigocóccica,Septicemia,Aids,Covid19,Hepatites virais,CA esôfago,CA estômago,...,Demais anomalias congênitas,Mal definidas,Acid trânsito e transporte terrestres,Quedas acidentais,Demais acidentes,Suicídios,Homicídios,Lesões intenc indeterminada,Demais causas externas,Demais causas de morte
0,Aricanduva/Formosa/Carrão,4,9.0,-,9.0,6.0,2,3,3,15.0,...,5,38.0,10.0,5,19.0,3,3,46.0,1,260.0
1,Butantã,5,10.0,1,12.0,12.0,3,2,8,20.0,...,4,62.0,13.0,10,23.0,6,10,66.0,3,292.0
2,Campo Limpo,7,16.0,-,17.0,23.0,7,7,19,34.0,...,12,119.0,30.0,15,22.0,9,18,104.0,8,459.0
3,Capela do Socorro,7,16.0,-,24.0,15.0,3,3,20,35.0,...,18,99.0,44.0,25,37.0,15,35,80.0,6,509.0
4,Casa Verde/Cachoeirinha,4,15.0,-,27.0,19.0,1,3,5,19.0,...,5,75.0,14.0,5,11.0,5,5,52.0,3,288.0
5,Cidade Ademar,-,20.0,2,13.0,14.0,3,1,14,26.0,...,9,78.0,12.0,8,18.0,14,9,75.0,3,312.0
6,Cidade Tiradentes,1,12.0,1,14.0,9.0,1,2,4,7.0,...,4,45.0,2.0,1,14.0,3,6,51.0,1,183.0
7,Ermelino Matarazzo,3,10.0,-,12.0,7.0,-,1,5,14.0,...,4,30.0,4.0,2,16.0,6,3,36.0,-,200.0
8,Freguesia/Brasilândia,6,20.0,-,23.0,16.0,2,3,11,25.0,...,7,97.0,19.0,13,26.0,5,8,80.0,2,353.0
9,Guaianases,1,16.0,-,16.0,12.0,1,1,4,13.0,...,2,55.0,2.0,4,10.0,-,5,68.0,3,255.0


Em seguida, "despivotamos" a tabela.

In [25]:
df_mortalidade_ajustado = df_mortalidade.melt(id_vars=['Subprefeitura residência'],
                                    var_name='Causa específica',
                                    value_name='Qtd óbitos')
df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos
0,Aricanduva/Formosa/Carrão,Doencas infec intest,4
1,Butantã,Doencas infec intest,5
2,Campo Limpo,Doencas infec intest,7
3,Capela do Socorro,Doencas infec intest,7
4,Casa Verde/Cachoeirinha,Doencas infec intest,4
...,...,...,...
2139,Sapopemba,Demais causas de morte,279.0
2140,Sé,Demais causas de morte,285.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,279.0
2142,Vila Mariana,Demais causas de morte,185.0


Por último, substituimos os `-` por 0.

In [26]:
hifen = df_mortalidade_ajustado['Qtd óbitos'].astype(str).str.contains('-')
df_mortalidade_ajustado.loc[hifen,'Qtd óbitos'] = 0
df_mortalidade_ajustado.loc[:,'Qtd óbitos'] = df_mortalidade_ajustado.loc[:,'Qtd óbitos'].astype(int)
df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos
0,Aricanduva/Formosa/Carrão,Doencas infec intest,4
1,Butantã,Doencas infec intest,5
2,Campo Limpo,Doencas infec intest,7
3,Capela do Socorro,Doencas infec intest,7
4,Casa Verde/Cachoeirinha,Doencas infec intest,4
...,...,...,...
2139,Sapopemba,Demais causas de morte,279
2140,Sé,Demais causas de morte,285
2141,Vila Maria/Vila Guilherme,Demais causas de morte,279
2142,Vila Mariana,Demais causas de morte,185


Posteriormente, cada causa específica dos óbitos será categorizado em Evitável, Tratável ou Evitável/Tratável, de acordo com uma tabela auxiliar definida pelo GT.

## Despesas

Para que seja possível relacionar as despesas com os outros indicadores, podemos usar os dados de regionalização do orçamento, disponíveis de acordo com o valor de detalhamento da ação. E para manter a completude dos dados, podemos complementar os dados regionalizados com os dados de execução não regionalizados.

Esses dados ficam disponíveis na [página de prestação de contas públicas da Secretaria Municipal da Fazenda](https://orcamento.sf.prefeitura.sp.gov.br/orcamento/execucao.php), na Base de Dados da Regionalização da Execução Orçamentária.

In [27]:
despesas_r_url = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2023/basedadosDA_1223.csv'
df_despesa_r = pd.read_csv(despesas_r_url,
                         encoding='latin1',
                         sep=';',
                         decimal=',')

df_despesa_r

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,DESCRIÇÃO_FONTE,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO
0,85,581,2023,2834,2023,2023-10-04 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,2063.85
1,85,581,2023,3292,2023,2023-11-09 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,2063.85
2,85,581,2023,3611,2023,2023-12-05 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,2063.85
3,85,582,2023,2231,2023,2023-08-03 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,2014.72
4,85,583,2023,2232,2023,2023-08-03 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,1965.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476493,85,579,2023,3607,2023,2023-12-05 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,2063.85
476494,85,580,2023,2229,2023,2023-08-03 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,2063.85
476495,85,581,2023,117,2024,2024-01-16 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,2063.85
476496,85,581,2023,2230,2023,2023-08-03 00:00:00.0000000,85,FTMSP,Fundação Theatro Municipal de São Paulo,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Supra Subprefeitura Centro,Supra-Distrital Centro,Despesa Regionalizável,2063.85


Primeiro, vamos filtrar os dados para manter apenas a função Saúde do orçamento.

In [28]:
df_despesa_r = df_despesa_r[df_despesa_r['DESCRIÇÃO_FUNÇÃO']=='Saúde']
df_despesa_r = df_despesa_r[df_despesa_r['ANO_LIQUIDAÇÃO']==2023]
df_despesa_r

,COD_EMPRESA_PMSP,COD_EMPENHO,ANO_EMPENHO,CÓDIGO_NLP,ANO_LIQUIDAÇÃO,DATA_LIQUIDAÇÃO,CÓDIGO_ÓRGÃO,SIGLA_ÓRGÃO,DESCRIÇÃO_ÓRGÃO,CÓDIGO_UNIDADE,...,DESCRIÇÃO_FONTE,CÓDIGO_EXERCÍCIO_FONTE,CÓDIGO_DESTINAÇÃO_RECURSO,CÓDIGO_VÍNCULO_PMSP,CÓDIGO_TIPO_CRÉDITO_ORÇAMENTÁRIO,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO
2879,1,138,2023,44535,2023,2023-03-02 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,357253.26
2880,1,138,2023,64466,2023,2023-03-24 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,351520.15
2881,1,138,2023,105742,2023,2023-05-10 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,359014.44
2895,1,140,2023,7780,2023,2023-01-19 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Norte,Subprefeitura Freguesia/Brasilândia,Supra-Distrital,Despesa Regionalizável,6182101.64
2896,1,140,2023,18932,2023,2023-02-01 00:00:00.0000000,84,FMS/SMS,Fundo Municipal de Saúde,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Norte,Subprefeitura Freguesia/Brasilândia,Supra-Distrital,Despesa Regionalizável,12776343.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
435745,4,5820,2023,8230,2023,2023-12-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,198500.10
435746,4,5821,2023,8231,2023,2023-12-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,13229486.42
435747,4,5822,2023,8232,2023,2023-12-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3317.54
435748,4,5823,2023,8233,2023,2023-12-29 00:00:00.0000000,2,HSPM,Hospital do Servidor Público Municipal,10,...,Recursos não vinculados de Impostos,1,500,9001,0,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,4423.39


Agora, agrupamos os dados pelas colunas que serão mantidas nos dados finais.

In [29]:
group_cols = ['CÓDIGO_SUBFUNÇÃO',
        'DESCRIÇÃO_SUBFUNÇÃO',
        'CÓDIGO_PROGRAMA',
        'DESCRIÇÃO_PROGRAMA',
        'REGIÃO',
        'SUBPREFEITURA',
        'DISTRITO',
        'TIPO_REGIONALIZAÇÃO']

measure_cols = ['VALOR_DETALHAMENTO_AÇÃO']

df_despesa_r_agrupado = df_despesa_r.groupby(group_cols).sum().round(2).reset_index().loc[:, group_cols + measure_cols]
df_despesa_r_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,VALOR_DETALHAMENTO_AÇÃO
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09
...,...,...,...,...,...,...,...,...,...
134,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,6.364286e+05
135,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06
136,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06
137,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05


Vamos ver quais valores aparecem como subfunção.

In [30]:
df_despesa_r_agrupado['DESCRIÇÃO_PROGRAMA'].value_counts()

DESCRIÇÃO_PROGRAMA
Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância      82
Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência    38
Ações e Serviços da Saúde Animal                                              10
Suporte Administrativo                                                         4
Modernização Tecnológica, Desburocratização e Inovação do Serviço Público      3
Benefícios e Previdência de Funcionários                                       1
Participação, Transparência e Controle Social da Administração Pública         1
Name: count, dtype: int64

Para os indicadores de "Despesas liquidadas em atenção básica e vigilância" e "Despesas liquidadas em atenção especializada e Suporte profilático terapêutico", parece que os programas serão o suficiente, com alguns não se encaixando em nenhuma das categorias.

Vamos agora obter os dados de despesas totais.

In [31]:
despesas_url = 'https://orcamento.sf.prefeitura.sp.gov.br/orcamento/uploads/2023/basedadosexecucao_1223.csv'
df_despesa = pd.read_csv(despesas_url,
                         encoding='latin1',
                         sep=';',
                         decimal=',')

df_despesa

,DataInicial,DataFinal,Cd_Exercicio,Cd_AnoExecucao,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
0,01/01/2023,31/12/2023,2023,2023,161750,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,23/01/2024
1,01/01/2023,31/12/2023,2023,2023,161749,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,23/01/2024
2,01/01/2023,31/12/2023,2023,2023,159818,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,23/01/2024
3,01/01/2023,31/12/2023,2023,2023,163083,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,8206727.38,6243403.30,6243403.30,4561508.41,4561508.41,1963324.08,23/01/2024
4,01/01/2023,31/12/2023,2023,2023,163084,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,10000000.00,8756657.96,8756657.96,8756657.96,6378324.65,1243342.04,23/01/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9158,01/01/2023,31/12/2023,2023,2023,159218,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0.0,0.0,0.0,414708.00,136967.98,136967.98,136967.98,118563.98,277740.02,23/01/2024
9159,01/01/2023,31/12/2023,2023,2023,159219,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,23/01/2024
9160,01/01/2023,31/12/2023,2023,2023,159220,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0.0,0.0,0.0,10000.00,0.00,0.00,0.00,0.00,10000.00,23/01/2024
9161,01/01/2023,31/12/2023,2023,2023,172372,Legislativo,77,FTCMSP,Fundo Especial de Despesas do Tribunal de Contas,10,...,0.0,0.0,0.0,450000.00,0.00,0.00,0.00,0.00,450000.00,23/01/2024


Primeiro, vamos filtrar os dados para manter apenas a função Saúde do orçamento.

In [32]:
df_despesa = df_despesa[df_despesa['Ds_Funcao']=='Saúde']
df_despesa = df_despesa[df_despesa['Cd_AnoExecucao']==2023]
df_despesa

,DataInicial,DataFinal,Cd_Exercicio,Cd_AnoExecucao,Cd_Dotacao_Id,Administracao,Cd_Orgao,Sigla_Orgao,Ds_Orgao,Cd_Unidade,...,Vl_Congelado,Vl_Descongelado,Vl_CongeladoLiquido,Disponivel,Vl_ReservadoLiquido,Vl_EmpenhadoLiquido,Vl_Liquidado,Vl_Pago,Saldo_Dotacao,DataExtracao
3,01/01/2023,31/12/2023,2023,2023,163083,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,8206727.38,6243403.30,6243403.30,4561508.41,4561508.41,1963324.08,23/01/2024
4,01/01/2023,31/12/2023,2023,2023,163084,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,10000000.00,8756657.96,8756657.96,8756657.96,6378324.65,1243342.04,23/01/2024
5,01/01/2023,31/12/2023,2023,2023,163086,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,10000000.00,3870279.58,3870279.58,3870279.58,2890236.45,6129720.42,23/01/2024
6,01/01/2023,31/12/2023,2023,2023,159819,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,23/01/2024
7,01/01/2023,31/12/2023,2023,2023,163085,Direta,7,FMD,Fundo Municipal de Desenvolvimento Social,10,...,0.0,0.0,0.0,55667612.83,25971315.21,25971315.21,2460055.22,2291542.01,29696297.62,23/01/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8324,01/01/2023,31/12/2023,2023,2023,170194,Indireta,2,HSPM,Hospital do Servidor Público Municipal,10,...,0.0,0.0,0.0,1500000.00,1346186.12,1346186.12,112076.02,102705.84,153813.88,23/01/2024
8325,01/01/2023,31/12/2023,2023,2023,160804,Indireta,2,HSPM,Hospital do Servidor Público Municipal,10,...,0.0,0.0,0.0,400000.00,318026.95,318026.95,179026.95,179026.95,81973.05,23/01/2024
8326,01/01/2023,31/12/2023,2023,2023,162624,Indireta,2,HSPM,Hospital do Servidor Público Municipal,10,...,200000.0,200000.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,23/01/2024
8327,01/01/2023,31/12/2023,2023,2023,162625,Indireta,2,HSPM,Hospital do Servidor Público Municipal,10,...,1000000.0,1000000.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,23/01/2024


Agora, agrupamos os dados pelas colunas que serão mantidas nos dados finais.

In [33]:
group_cols = ['Cd_SubFuncao',
        'Ds_SubFuncao',
        'Cd_Programa',
        'Ds_Programa']

measure_cols = ['Vl_Liquidado']

df_despesa_agrupado = df_despesa.groupby(group_cols).sum().round(2).reset_index().loc[:, group_cols + measure_cols]
df_despesa_agrupado

,Cd_SubFuncao,Ds_SubFuncao,Cd_Programa,Ds_Programa,Vl_Liquidado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",1.562793e+07
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",5.392435e+06
2,122,Administração Geral,3012,"Participação, Transparência e Controle Social ...",0.000000e+00
3,122,Administração Geral,3024,Suporte Administrativo,2.594339e+09
4,126,Tecnologia da Informação,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",2.352275e+07
5,126,Tecnologia da Informação,3011,"Modernização Tecnológica, Desburocratização e ...",0.000000e+00
6,126,Tecnologia da Informação,3024,Suporte Administrativo,2.213463e+08
7,128,Formação de Recursos Humanos,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",2.860600e+05
8,128,Formação de Recursos Humanos,3011,"Modernização Tecnológica, Desburocratização e ...",2.253935e+05
9,131,Comunicação Social,3012,"Participação, Transparência e Controle Social ...",0.000000e+00


Baixados todos os dados, vamos começar as transformações para adequar as categorias dos dados às necessidades dos indicadores.

# Transformação e padronização

Primeiro, vamos criar as variáveis que representarão os níveis de atenção padronizados.

In [34]:
NIVEL_COL_NAME = 'Nível de atenção padronizado'
NIVEL_AB_VIGILANCIA = 'Atenção Básica e Vigilância'
NIVEL_ESP = 'Atenção Especializada e Suporte profilático terapêutico'
NIVEL_OUTROS = 'Outros níveis de atenção'

Nas próximas seções, vamos também padronizar os nomes das Subprefeituras, com uma função já disponível no módulo `core.utils.cleaning`.

## Consultas médicas

In [35]:
df_consultas_ajustado

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363
...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951


In [36]:
subpref_map = {
    "Aricanduva/Formosa/Carrão": "ARICANDUVA-FORMOSA-CARRAO",
    "Butantã": "BUTANTA",
    "Campo Limpo": "CAMPO LIMPO",
    "Capela do Socorro": "CAPELA DO SOCORRO",
    "Casa Verde/Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
    "Cidade Ademar": "CIDADE ADEMAR",
    "Cidade Tiradentes": "CIDADE TIRADENTES",
    "Ermelino Matarazzo": "ERMELINO MATARAZZO",
    "Freguesia/Brasilândia": "FREGUESIA-BRASILANDIA",
    "Guaianases": "GUAIANASES",
    "Ipiranga": "IPIRANGA",
    "Itaim Paulista": "ITAIM PAULISTA",
    "Itaquera": "ITAQUERA",
    "Jabaquara": "JABAQUARA",
    "Jaçanã/Tremembé": "JACANA-TREMEMBE",
    "Lapa": "LAPA",
    "M'Boi Mirim": "M BOI MIRIM",
    "Mooca": "MOOCA",
    "Parelheiros": "PARELHEIROS",
    "Penha": "PENHA",
    "Perus": "PERUS",
    "Pinheiros": "PINHEIROS",
    "Pirituba/Jaraguá": "PIRITUBA-JARAGUA",
    "Santana/Tucuruvi": "SANTANA-TUCURUVI",
    "Santo Amaro": "SANTO AMARO",
    "São Mateus": "SAO MATEUS",
    "São Miguel Paulista": "SAO MIGUEL",
    "Sapopemba": "SAPOPEMBA",
    "Sé": "SE",
    "Vila Maria/Vila Guilherme": "VILA MARIA-VILA GUILHERME",
    "Vila Mariana": "VILA MARIANA",
    "Vila Prudente": "VILA PRUDENTE"
}

In [37]:
df_consultas_ajustado.loc[:, 'SUBPREFEITURA_QLIK'] = (
    df_consultas_ajustado.loc[:, 'Subprefeitura'].map(subpref_map)
)

df_consultas_ajustado

/tmp/ipykernel_5635/2884928701.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_consultas_ajustado.loc[:, 'SUBPREFEITURA_QLIK'] = (


,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas,SUBPREFEITURA_QLIK
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894,ARICANDUVA-FORMOSA-CARRAO
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917,BUTANTA
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251,CAMPO LIMPO
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063,CAPELA DO SOCORRO
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363,CASA VERDE-CACHOEIRINHA
...,...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822,SAPOPEMBA
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857,SE
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0,VILA MARIA-VILA GUILHERME
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951,VILA MARIANA


In [38]:
df_consultas_ajustado['Categoria'].unique()

array(['Consulta Médica na Atenção Básica',
       'Consulta Médica\nna Atenção Especializada',
       'Consulta Médica/Atendimento em Urgência/Emergência'], dtype=object)

In [39]:
df_consultas_final = df_consultas_ajustado.copy()

filtro_ab = df_consultas_final['Categoria'].str.contains('Atenção Básica')

df_consultas_final.loc[filtro_ab, NIVEL_COL_NAME] = NIVEL_AB_VIGILANCIA
df_consultas_final.loc[~filtro_ab, NIVEL_COL_NAME] = NIVEL_ESP

df_consultas_final

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas,SUBPREFEITURA_QLIK,Nível de atenção padronizado
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894,ARICANDUVA-FORMOSA-CARRAO,Atenção Básica e Vigilância
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917,BUTANTA,Atenção Básica e Vigilância
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251,CAMPO LIMPO,Atenção Básica e Vigilância
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063,CAPELA DO SOCORRO,Atenção Básica e Vigilância
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363,CASA VERDE-CACHOEIRINHA,Atenção Básica e Vigilância
...,...,...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822,SAPOPEMBA,Atenção Especializada e Suporte profilático te...
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857,SE,Atenção Especializada e Suporte profilático te...
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0,VILA MARIA-VILA GUILHERME,Atenção Especializada e Suporte profilático te...
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951,VILA MARIANA,Atenção Especializada e Suporte profilático te...


In [40]:
df_consultas_final['subprefeitura_nivel'] = (
    df_consultas_final['SUBPREFEITURA_QLIK'] +
    ' | ' +
    df_consultas_final['Nível de atenção padronizado']
)

df_consultas_final

,Subprefeitura,Categoria,Subcategoria,Qtd_Consultas,SUBPREFEITURA_QLIK,Nível de atenção padronizado,subprefeitura_nivel
0,Aricanduva/Formosa/Carrão,Consulta Médica na Atenção Básica,Não Urgência\nSMS,188894,ARICANDUVA-FORMOSA-CARRAO,Atenção Básica e Vigilância,ARICANDUVA-FORMOSA-CARRAO | Atenção Básica e V...
1,Butantã,Consulta Médica na Atenção Básica,Não Urgência\nSMS,443917,BUTANTA,Atenção Básica e Vigilância,BUTANTA | Atenção Básica e Vigilância
2,Campo Limpo,Consulta Médica na Atenção Básica,Não Urgência\nSMS,837251,CAMPO LIMPO,Atenção Básica e Vigilância,CAMPO LIMPO | Atenção Básica e Vigilância
3,Capela do Socorro,Consulta Médica na Atenção Básica,Não Urgência\nSMS,554063,CAPELA DO SOCORRO,Atenção Básica e Vigilância,CAPELA DO SOCORRO | Atenção Básica e Vigilância
4,Casa Verde/Cachoeirinha,Consulta Médica na Atenção Básica,Não Urgência\nSMS,327363,CASA VERDE-CACHOEIRINHA,Atenção Básica e Vigilância,CASA VERDE-CACHOEIRINHA | Atenção Básica e Vig...
...,...,...,...,...,...,...,...
187,Sapopemba,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,21822,SAPOPEMBA,Atenção Especializada e Suporte profilático te...,SAPOPEMBA | Atenção Especializada e Suporte pr...
188,Sé,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,261857,SE,Atenção Especializada e Suporte profilático te...,SE | Atenção Especializada e Suporte profiláti...
189,Vila Maria/Vila Guilherme,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,0,VILA MARIA-VILA GUILHERME,Atenção Especializada e Suporte profilático te...,VILA MARIA-VILA GUILHERME | Atenção Especializ...
190,Vila Mariana,Consulta Médica/Atendimento em Urgência/Emergê...,Gestão \nSES,139951,VILA MARIANA,Atenção Especializada e Suporte profilático te...,VILA MARIANA | Atenção Especializada e Suporte...


## Leitos de internação e complementares

Primeiro, vamos organizar os leitos nas categorias: Leitos de internações; Leitos complementares.

A regra específicada pelo GT foi a seguinte:

 - Leitos de internações são todas as categorias de leito, exceto as categorias `3-Complementar` e `7-Hospital/DIA`;

 - Leitos complementares são os da categoria `3-Complementar`;

In [41]:
df_leito_ajustado['Categoria específica'].value_counts()

Categoria específica
1-Cirúrgico                32
2-Clínico                  32
3-Complementar             32
4-Obstétrico               32
5-Pediátrico               32
6-Outras Especialidades    32
7-Hospital/DIA             32
Name: count, dtype: int64

In [42]:
leito_map = {
    '1-Cirúrgico': 'Leitos de internação',
    '2-Clínico': 'Leitos de internação',
    '3-Complementar': 'Leitos complementares',
    '4-Obstétrico': 'Leitos de internação',
    '5-Pediátrico': 'Leitos de internação',
    '6-Outras Especialidades': 'Leitos de internação',
    '7-Hospital/DIA': None,
}

df_leito_ajustado.loc[:, 'Categoria geral'] = df_leito_ajustado['Categoria específica'].map(leito_map)
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos,Categoria geral
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0,Leitos de internação
1,Butantã,1-Cirúrgico,78,Leitos de internação
2,Campo Limpo,1-Cirúrgico,0,Leitos de internação
3,Capela do Socorro,1-Cirúrgico,78,Leitos de internação
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86,Leitos de internação
...,...,...,...,...
219,Sapopemba,7-Hospital/DIA,6,None
220,Sé,7-Hospital/DIA,69,None
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6,None
222,Vila Mariana,7-Hospital/DIA,166,None


Agora, adicionamos também a classificação geral por nível de atenção (básica ou especializada). Para leitos, todos são utilizados na Atenção especializada, então é só adicionarmos uma coluna com o valor como padrão.

In [43]:
df_leito_ajustado.loc[:, NIVEL_COL_NAME] = NIVEL_ESP
df_leito_ajustado

,Pref Regional,Categoria específica,Qtd leitos,Categoria geral,Nível de atenção padronizado
0,Aricanduva/ Formosa/ Carrão,1-Cirúrgico,0,Leitos de internação,Atenção Especializada e Suporte profilático te...
1,Butantã,1-Cirúrgico,78,Leitos de internação,Atenção Especializada e Suporte profilático te...
2,Campo Limpo,1-Cirúrgico,0,Leitos de internação,Atenção Especializada e Suporte profilático te...
3,Capela do Socorro,1-Cirúrgico,78,Leitos de internação,Atenção Especializada e Suporte profilático te...
4,Casa Verde/ Cachoeirinha,1-Cirúrgico,86,Leitos de internação,Atenção Especializada e Suporte profilático te...
...,...,...,...,...,...
219,Sapopemba,7-Hospital/DIA,6,None,Atenção Especializada e Suporte profilático te...
220,Sé,7-Hospital/DIA,69,None,Atenção Especializada e Suporte profilático te...
221,Vila Maria/ Vila Guilherme,7-Hospital/DIA,6,None,Atenção Especializada e Suporte profilático te...
222,Vila Mariana,7-Hospital/DIA,166,None,Atenção Especializada e Suporte profilático te...


In [44]:
group_cols = ['Pref Regional',
              'Categoria geral',
              NIVEL_COL_NAME]

df_leitos_final = (
    df_leito_ajustado[group_cols + ['Qtd leitos']]
    .groupby(group_cols)
    .sum()
    .reset_index()
)

df_leitos_final

,Pref Regional,Categoria geral,Nível de atenção padronizado,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55


In [45]:
df_leitos_final = df_leitos_final.rename(columns={'Pref Regional': 'Subprefeitura'})
df_leitos_final

,Subprefeitura,Categoria geral,Nível de atenção padronizado,Qtd leitos
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0
...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55


In [46]:
subpref_map = {
    "Aricanduva/ Formosa/ Carrão": "ARICANDUVA-FORMOSA-CARRAO",
    "Butantã": "BUTANTA",
    "Campo Limpo": "CAMPO LIMPO",
    "Capela do Socorro": "CAPELA DO SOCORRO",
    "Casa Verde/ Cachoeirinha": "CASA VERDE-CACHOEIRINHA",
    "Cidade Ademar": "CIDADE ADEMAR",
    "Cidade Tiradentes": "CIDADE TIRADENTES",
    "Ermelino Matarazzo": "ERMELINO MATARAZZO",
    "Freguesia/ Brasilândia": "FREGUESIA-BRASILANDIA",
    "Guaianases": "GUAIANASES",
    "Ipiranga": "IPIRANGA",
    "Itaim Paulista": "ITAIM PAULISTA",
    "Itaquera": "ITAQUERA",
    "Jabaquara": "JABAQUARA",
    "Jaçanã/ Tremembé": "JACANA-TREMEMBE",
    "Lapa": "LAPA",
    "M Boi Mirim": "M BOI MIRIM",
    "Mooca": "MOOCA",
    "Parelheiros": "PARELHEIROS",
    "Penha": "PENHA",
    "Perus": "PERUS",
    "Pinheiros": "PINHEIROS",
    "Pirituba/ Jaraguá": "PIRITUBA-JARAGUA",
    "Santana/ Tucuruvi": "SANTANA-TUCURUVI",
    "Santo Amaro": "SANTO AMARO",
    "Sapopemba": "SAPOPEMBA",
    "São Mateus": "SAO MATEUS",
    "São Miguel": "SAO MIGUEL",
    "Sé": "SE",
    "Vila Maria/ Vila Guilherme": "VILA MARIA-VILA GUILHERME",
    "Vila Mariana": "VILA MARIANA",
    "Vila Prudente": "VILA PRUDENTE"
}


In [47]:
df_leitos_final.loc[:, 'SUBPREFEITURA_QLIK'] = (
    df_leitos_final.loc[:, 'Subprefeitura'].map(subpref_map)
)

df_leitos_final

,Subprefeitura,Categoria geral,Nível de atenção padronizado,Qtd leitos,SUBPREFEITURA_QLIK
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0,ARICANDUVA-FORMOSA-CARRAO
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95,ARICANDUVA-FORMOSA-CARRAO
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66,BUTANTA
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344,BUTANTA
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0,CAMPO LIMPO
...,...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144,VILA MARIA-VILA GUILHERME
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324,VILA MARIANA
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491,VILA MARIANA
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55,VILA PRUDENTE


In [48]:
df_leitos_final['subprefeitura_nivel'] = (
    df_leitos_final['SUBPREFEITURA_QLIK'] +
    ' | ' +
    df_leitos_final['Nível de atenção padronizado']
)

df_leitos_final

,Subprefeitura,Categoria geral,Nível de atenção padronizado,Qtd leitos,SUBPREFEITURA_QLIK,subprefeitura_nivel
0,Aricanduva/ Formosa/ Carrão,Leitos complementares,Atenção Especializada e Suporte profilático te...,0,ARICANDUVA-FORMOSA-CARRAO,ARICANDUVA-FORMOSA-CARRAO | Atenção Especializ...
1,Aricanduva/ Formosa/ Carrão,Leitos de internação,Atenção Especializada e Suporte profilático te...,95,ARICANDUVA-FORMOSA-CARRAO,ARICANDUVA-FORMOSA-CARRAO | Atenção Especializ...
2,Butantã,Leitos complementares,Atenção Especializada e Suporte profilático te...,66,BUTANTA,BUTANTA | Atenção Especializada e Suporte prof...
3,Butantã,Leitos de internação,Atenção Especializada e Suporte profilático te...,344,BUTANTA,BUTANTA | Atenção Especializada e Suporte prof...
4,Campo Limpo,Leitos complementares,Atenção Especializada e Suporte profilático te...,0,CAMPO LIMPO,CAMPO LIMPO | Atenção Especializada e Suporte ...
...,...,...,...,...,...,...
59,Vila Maria/ Vila Guilherme,Leitos de internação,Atenção Especializada e Suporte profilático te...,144,VILA MARIA-VILA GUILHERME,VILA MARIA-VILA GUILHERME | Atenção Especializ...
60,Vila Mariana,Leitos complementares,Atenção Especializada e Suporte profilático te...,324,VILA MARIANA,VILA MARIANA | Atenção Especializada e Suporte...
61,Vila Mariana,Leitos de internação,Atenção Especializada e Suporte profilático te...,1491,VILA MARIANA,VILA MARIANA | Atenção Especializada e Suporte...
62,Vila Prudente,Leitos complementares,Atenção Especializada e Suporte profilático te...,55,VILA PRUDENTE,VILA PRUDENTE | Atenção Especializada e Suport...


## Despesas

Primeiro, vamos complementar os dados de despesas regionalizados com os não regionalizados. Para isso, precisamos subtrair os dados de despesas detalhados dos dado de despesa não detalhados e verificar quanto dos dados não está na tabela de detalhamento.

In [49]:
df_despesa_r_agg = (
    df_despesa_r_agrupado
    .groupby(['CÓDIGO_SUBFUNÇÃO','CÓDIGO_PROGRAMA'])
    .agg({'VALOR_DETALHAMENTO_AÇÃO': 'sum'})
    .round(2)
    .reset_index()
)

df_despesa_ajustado = df_despesa_agrupado.merge(
    df_despesa_r_agg,
    left_on=['Cd_SubFuncao', 'Cd_Programa'],
    right_on=['CÓDIGO_SUBFUNÇÃO', 'CÓDIGO_PROGRAMA'],
    how='left'
).drop(columns=['CÓDIGO_SUBFUNÇÃO', 'CÓDIGO_PROGRAMA'])

df_despesa_ajustado

,Cd_SubFuncao,Ds_SubFuncao,Cd_Programa,Ds_Programa,Vl_Liquidado,VALOR_DETALHAMENTO_AÇÃO
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",1.562793e+07,1.562793e+07
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",5.392435e+06,5.392435e+06
2,122,Administração Geral,3012,"Participação, Transparência e Controle Social ...",0.000000e+00,NaN
3,122,Administração Geral,3024,Suporte Administrativo,2.594339e+09,2.136985e+09
4,126,Tecnologia da Informação,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",2.352275e+07,2.352275e+07
5,126,Tecnologia da Informação,3011,"Modernização Tecnológica, Desburocratização e ...",0.000000e+00,NaN
6,126,Tecnologia da Informação,3024,Suporte Administrativo,2.213463e+08,2.210844e+08
7,128,Formação de Recursos Humanos,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",2.860600e+05,2.860600e+05
8,128,Formação de Recursos Humanos,3011,"Modernização Tecnológica, Desburocratização e ...",2.253935e+05,1.942378e+05
9,131,Comunicação Social,3012,"Participação, Transparência e Controle Social ...",0.000000e+00,NaN


In [50]:
df_despesa_ajustado.loc[:, 'Vl_Liquidado_N_Detalhado'] = (
    df_despesa_ajustado.loc[:, 'Vl_Liquidado']
    - df_despesa_ajustado.loc[:, 'VALOR_DETALHAMENTO_AÇÃO']
).round(2)

df_despesa_ajustado

,Cd_SubFuncao,Ds_SubFuncao,Cd_Programa,Ds_Programa,Vl_Liquidado,VALOR_DETALHAMENTO_AÇÃO,Vl_Liquidado_N_Detalhado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",1.562793e+07,1.562793e+07,0.000000e+00
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",5.392435e+06,5.392435e+06,0.000000e+00
2,122,Administração Geral,3012,"Participação, Transparência e Controle Social ...",0.000000e+00,NaN,NaN
3,122,Administração Geral,3024,Suporte Administrativo,2.594339e+09,2.136985e+09,4.573547e+08
4,126,Tecnologia da Informação,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",2.352275e+07,2.352275e+07,0.000000e+00
5,126,Tecnologia da Informação,3011,"Modernização Tecnológica, Desburocratização e ...",0.000000e+00,NaN,NaN
6,126,Tecnologia da Informação,3024,Suporte Administrativo,2.213463e+08,2.210844e+08,2.618496e+05
7,128,Formação de Recursos Humanos,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",2.860600e+05,2.860600e+05,0.000000e+00
8,128,Formação de Recursos Humanos,3011,"Modernização Tecnológica, Desburocratização e ...",2.253935e+05,1.942378e+05,3.115574e+04
9,131,Comunicação Social,3012,"Participação, Transparência e Controle Social ...",0.000000e+00,NaN,NaN


In [51]:
df_despesa_ajustado[df_despesa_ajustado['Vl_Liquidado_N_Detalhado']<0]

,Cd_SubFuncao,Ds_SubFuncao,Cd_Programa,Ds_Programa,Vl_Liquidado,VALOR_DETALHAMENTO_AÇÃO,Vl_Liquidado_N_Detalhado


Como todos os valores de liquidação não detalhada são maiores do que zero, não parece haver nenhum problema com as agregação. Então, vamos adicionar os dados não detalhados ao dataframe de orçamento regionalizado.

In [52]:
despesa_cols_map = {'Cd_SubFuncao': 'CÓDIGO_SUBFUNÇÃO',
                      'Ds_SubFuncao': 'DESCRIÇÃO_SUBFUNÇÃO',
                      'Cd_Programa': 'CÓDIGO_PROGRAMA',
                      'Ds_Programa': 'DESCRIÇÃO_PROGRAMA',
                      'Vl_Liquidado_N_Detalhado': 'Vl_Liquidado'}

df_despesa_ajustado = (
    df_despesa_ajustado
    .drop(columns=['VALOR_DETALHAMENTO_AÇÃO', 'Vl_Liquidado'])
    .rename(columns=despesa_cols_map)
    )

df_despesa_ajustado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,Vl_Liquidado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",0.000000e+00
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",0.000000e+00
2,122,Administração Geral,3012,"Participação, Transparência e Controle Social ...",NaN
3,122,Administração Geral,3024,Suporte Administrativo,4.573547e+08
4,126,Tecnologia da Informação,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",0.000000e+00
5,126,Tecnologia da Informação,3011,"Modernização Tecnológica, Desburocratização e ...",NaN
6,126,Tecnologia da Informação,3024,Suporte Administrativo,2.618496e+05
7,128,Formação de Recursos Humanos,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",0.000000e+00
8,128,Formação de Recursos Humanos,3011,"Modernização Tecnológica, Desburocratização e ...",3.115574e+04
9,131,Comunicação Social,3012,"Participação, Transparência e Controle Social ...",NaN


In [53]:
df_despesa_r_agrupado = (
    df_despesa_r_agrupado
    .rename(columns={'VALOR_DETALHAMENTO_AÇÃO': 'Vl_Liquidado'})
)

df_despesa_r_agrupado

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09
...,...,...,...,...,...,...,...,...,...
134,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Norte,Subprefeitura Santana/Tucuruvi,Supra-Distrital,Despesa Regionalizável,6.364286e+05
135,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Butantã,Supra-Distrital,Despesa Regionalizável,4.470286e+06
136,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Oeste,Subprefeitura Pinheiros,Supra-Distrital,Despesa Regionalizável,3.244937e+06
137,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,Sul,Subprefeitura Santo Amaro,Supra-Distrital,Despesa Regionalizável,5.500000e+05


In [54]:
df_despesas_final = pd.concat([df_despesa_r_agrupado,
                               df_despesa_ajustado], ignore_index=True)

df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09
...,...,...,...,...,...,...,...,...,...
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06
156,304,Vigilância Sanitária,3027,Ações e Serviços da Saúde Animal,NaN,NaN,NaN,NaN,0.000000e+00


Finalmente, removemos as linhas sem liquidação.

In [55]:
df_despesas_final = df_despesas_final.loc[~df_despesas_final['Vl_Liquidado'].isna()]
df_despesas_final = df_despesas_final.loc[df_despesas_final['Vl_Liquidado']>0]

df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09
...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06


Para manter mais flexibilidade, vamos manter todas as colunas presentes na tabela de despesas, mesmo que usemos apenas a coluna de programa. Então, o que vamos fazer aqui é criar a nova coluna de nível de atenção padronizado com base nos programas.

In [56]:
df_despesas_final['DESCRIÇÃO_PROGRAMA'].value_counts()

DESCRIÇÃO_PROGRAMA
Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância      86
Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência    40
Ações e Serviços da Saúde Animal                                              10
Suporte Administrativo                                                         6
Modernização Tecnológica, Desburocratização e Inovação do Serviço Público      4
Benefícios e Previdência de Funcionários                                       2
Participação, Transparência e Controle Social da Administração Pública         2
Name: count, dtype: int64

In [57]:
map_despesa = {
    'Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilância': NIVEL_AB_VIGILANCIA,
    'Ações e Serviços da Saúde em Atenção Hospitalar e de Urgência e Emergência': NIVEL_ESP
}

df_despesas_final.loc[:, NIVEL_COL_NAME] = df_despesas_final.loc[:, 'DESCRIÇÃO_PROGRAMA'].map(map_despesa)
df_despesas_final.loc[:, NIVEL_COL_NAME] = df_despesas_final.loc[:, NIVEL_COL_NAME].fillna(NIVEL_OUTROS)
df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado,Nível de atenção padronizado
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07,Atenção Básica e Vigilância
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06,Outros níveis de atenção
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06,Outros níveis de atenção
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08,Outros níveis de atenção
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09,Outros níveis de atenção
...,...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08,Atenção Especializada e Suporte profilático te...
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05,Atenção Básica e Vigilância
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03,Atenção Especializada e Suporte profilático te...
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06,Atenção Básica e Vigilância


Vamos também criar uma nova coluna com o nome da subprefeitura, de acordo com o padrão existente no arquivo com os polígonos do Qlik.

In [58]:
subpref_filter = (
    df_despesas_final.loc[:, 'SUBPREFEITURA']
    .str.startswith('Subprefeitura ')
    .fillna(False)
)

df_despesas_final.loc[subpref_filter, 'SUBPREFEITURA_QLIK'] = (
    df_despesas_final
    .loc[subpref_filter, 'SUBPREFEITURA']
    .str.removeprefix('Subprefeitura ')
)

df_despesas_final

/tmp/ipykernel_5635/3367335398.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .fillna(False)


,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado,Nível de atenção padronizado,SUBPREFEITURA_QLIK
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07,Atenção Básica e Vigilância,NaN
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06,Outros níveis de atenção,Sé
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06,Outros níveis de atenção,NaN
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08,Outros níveis de atenção,Sé
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09,Outros níveis de atenção,NaN
...,...,...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08,Atenção Especializada e Suporte profilático te...,NaN
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05,Atenção Básica e Vigilância,NaN
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03,Atenção Especializada e Suporte profilático te...,NaN
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06,Atenção Básica e Vigilância,NaN


In [59]:
subpref_map = {
    'Aricanduva/Formosa/Carrão': 'ARICANDUVA-FORMOSA-CARRAO',
    'Butantã': 'BUTANTA',
    'Campo Limpo': 'CAMPO LIMPO',
    'Capela do Socorro': 'CAPELA DO SOCORRO',
    'Casa Verde/Cachoeirinha': 'CASA VERDE-CACHOEIRINHA',
    'Cidade Ademar': 'CIDADE ADEMAR',
    'Cidade Tiradentes': 'CIDADE TIRADENTES',
    'Ermelino Matarazzo': 'ERMELINO MATARAZZO',
    'Freguesia/Brasilândia': 'FREGUESIA-BRASILANDIA',
    'Ipiranga': 'IPIRANGA',
    'Itaim Paulista': 'ITAIM PAULISTA',
    'Itaquera': 'ITAQUERA',
    'Jabaquara': 'JABAQUARA',
    'Jaçanã/Tremembé': 'JACANA-TREMEMBE',
    'Lapa': 'LAPA',
    "M'Boi Mirim": 'M BOI MIRIM',
    'Mooca': 'MOOCA',
    'Parelheiros': 'PARELHEIROS',
    'Penha': 'PENHA',
    'Perus/Anhanguera': 'PERUS',
    'Pinheiros': 'PINHEIROS',
    'Pirituba/Jaraguá': 'PIRITUBA-JARAGUA',
    'Santana/Tucuruvi': 'SANTANA-TUCURUVI',
    'Santo Amaro': 'SANTO AMARO',
    'Sapopemba': 'SAPOPEMBA',
    'São Mateus': 'SAO MATEUS',
    'São Miguel Paulista': 'SAO MIGUEL',
    'Sé': 'SE',
    'Vila Maria/Vila Guilherme': 'VILA MARIA-VILA GUILHERME',
    'Vila Mariana': 'VILA MARIANA',
    'de Guaianases': 'GUAIANASES',
    'de Vila Prudente': 'VILA PRUDENTE'
}

In [60]:
df_despesas_final.loc[subpref_filter, 'SUBPREFEITURA_QLIK'] = (
    df_despesas_final.loc[subpref_filter, 'SUBPREFEITURA_QLIK'].map(subpref_map)
)

df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado,Nível de atenção padronizado,SUBPREFEITURA_QLIK
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07,Atenção Básica e Vigilância,NaN
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06,Outros níveis de atenção,SE
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06,Outros níveis de atenção,NaN
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08,Outros níveis de atenção,SE
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09,Outros níveis de atenção,NaN
...,...,...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08,Atenção Especializada e Suporte profilático te...,NaN
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05,Atenção Básica e Vigilância,NaN
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03,Atenção Especializada e Suporte profilático te...,NaN
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06,Atenção Básica e Vigilância,NaN


In [61]:
df_despesas_final['subprefeitura_nivel'] = (
    df_despesas_final['SUBPREFEITURA_QLIK'] +
    ' | ' +
    df_despesas_final['Nível de atenção padronizado']
)

df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado,Nível de atenção padronizado,SUBPREFEITURA_QLIK,subprefeitura_nivel
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07,Atenção Básica e Vigilância,NaN,NaN
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06,Outros níveis de atenção,SE,SE | Outros níveis de atenção
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06,Outros níveis de atenção,NaN,NaN
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08,Outros níveis de atenção,SE,SE | Outros níveis de atenção
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09,Outros níveis de atenção,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08,Atenção Especializada e Suporte profilático te...,NaN,NaN
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05,Atenção Básica e Vigilância,NaN,NaN
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03,Atenção Especializada e Suporte profilático te...,NaN,NaN
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06,Atenção Básica e Vigilância,NaN,NaN


Por último, vamos criar uma coluna para classificar o nível de regionalização.

In [62]:
df_despesas_final.loc[
    ~df_despesas_final['REGIÃO'].str.contains('Supra', na=False),
    'NIVEL_REGIONALIZAÇÃO'] = 'Região'

df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado,Nível de atenção padronizado,SUBPREFEITURA_QLIK,subprefeitura_nivel,NIVEL_REGIONALIZAÇÃO
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07,Atenção Básica e Vigilância,NaN,NaN,NaN
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06,Outros níveis de atenção,SE,SE | Outros níveis de atenção,Região
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06,Outros níveis de atenção,NaN,NaN,NaN
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08,Outros níveis de atenção,SE,SE | Outros níveis de atenção,Região
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09,Outros níveis de atenção,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08,Atenção Especializada e Suporte profilático te...,NaN,NaN,Região
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05,Atenção Básica e Vigilância,NaN,NaN,Região
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03,Atenção Especializada e Suporte profilático te...,NaN,NaN,Região
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06,Atenção Básica e Vigilância,NaN,NaN,Região


In [63]:
df_despesas_final.loc[
    ~df_despesas_final['SUBPREFEITURA'].str.contains('Supra', na=False),
    'NIVEL_REGIONALIZAÇÃO'] = 'Subprefeitura'

df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado,Nível de atenção padronizado,SUBPREFEITURA_QLIK,subprefeitura_nivel,NIVEL_REGIONALIZAÇÃO
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07,Atenção Básica e Vigilância,NaN,NaN,NaN
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06,Outros níveis de atenção,SE,SE | Outros níveis de atenção,Subprefeitura
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06,Outros níveis de atenção,NaN,NaN,NaN
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08,Outros níveis de atenção,SE,SE | Outros níveis de atenção,Subprefeitura
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09,Outros níveis de atenção,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08,Atenção Especializada e Suporte profilático te...,NaN,NaN,Subprefeitura
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05,Atenção Básica e Vigilância,NaN,NaN,Subprefeitura
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03,Atenção Especializada e Suporte profilático te...,NaN,NaN,Subprefeitura
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06,Atenção Básica e Vigilância,NaN,NaN,Subprefeitura


In [64]:
df_despesas_final.loc[df_despesas_final['TIPO_REGIONALIZAÇÃO']=='Despesa Não-Regionalizável', 'NIVEL_REGIONALIZAÇÃO'] = 'Não regionalizável'
df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado,Nível de atenção padronizado,SUBPREFEITURA_QLIK,subprefeitura_nivel,NIVEL_REGIONALIZAÇÃO
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07,Atenção Básica e Vigilância,NaN,NaN,Não regionalizável
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06,Outros níveis de atenção,SE,SE | Outros níveis de atenção,Subprefeitura
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06,Outros níveis de atenção,NaN,NaN,Não regionalizável
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08,Outros níveis de atenção,SE,SE | Outros níveis de atenção,Subprefeitura
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09,Outros níveis de atenção,NaN,NaN,Não regionalizável
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08,Atenção Especializada e Suporte profilático te...,NaN,NaN,Subprefeitura
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05,Atenção Básica e Vigilância,NaN,NaN,Subprefeitura
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03,Atenção Especializada e Suporte profilático te...,NaN,NaN,Subprefeitura
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06,Atenção Básica e Vigilância,NaN,NaN,Subprefeitura


In [65]:
df_despesas_final.loc[df_despesas_final['NIVEL_REGIONALIZAÇÃO'].isna(), 'NIVEL_REGIONALIZAÇÃO'] = 'Não detalhado'
df_despesas_final

,CÓDIGO_SUBFUNÇÃO,DESCRIÇÃO_SUBFUNÇÃO,CÓDIGO_PROGRAMA,DESCRIÇÃO_PROGRAMA,REGIÃO,SUBPREFEITURA,DISTRITO,TIPO_REGIONALIZAÇÃO,Vl_Liquidado,Nível de atenção padronizado,SUBPREFEITURA_QLIK,subprefeitura_nivel,NIVEL_REGIONALIZAÇÃO
0,122,Administração Geral,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.562793e+07,Atenção Básica e Vigilância,NaN,NaN,Não regionalizável
1,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,1.084279e+06,Outros níveis de atenção,SE,SE | Outros níveis de atenção,Subprefeitura
2,122,Administração Geral,3011,"Modernização Tecnológica, Desburocratização e ...",Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,4.308156e+06,Outros níveis de atenção,NaN,NaN,Não regionalizável
3,122,Administração Geral,3024,Suporte Administrativo,Centro,Subprefeitura Sé,Supra-Distrital,Despesa Regionalizável,3.228108e+08,Outros níveis de atenção,SE,SE | Outros níveis de atenção,Subprefeitura
4,122,Administração Geral,3024,Suporte Administrativo,Supra-Regional,Supra Subprefeitura,Supra-Distrital,Despesa Não-Regionalizável,1.814174e+09,Outros níveis de atenção,NaN,NaN,Não regionalizável
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,302,Assistência Hospitalar e Ambulatorial,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,2.707455e+08,Atenção Especializada e Suporte profilático te...,NaN,NaN,Subprefeitura
153,303,Suporte Profilático e Terapêutico,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,4.501761e+05,Atenção Básica e Vigilância,NaN,NaN,Subprefeitura
154,303,Suporte Profilático e Terapêutico,3026,Ações e Serviços da Saúde em Atenção Hospitala...,NaN,NaN,NaN,NaN,6.047360e+03,Atenção Especializada e Suporte profilático te...,NaN,NaN,Subprefeitura
155,304,Vigilância Sanitária,3003,"Ações e Serviços da Saúde em Atenção Básica, E...",NaN,NaN,NaN,NaN,8.439791e+06,Atenção Básica e Vigilância,NaN,NaN,Subprefeitura


## Mortalidade

In [66]:
subpref_map = {
    'Aricanduva/Formosa/Carrão': 'ARICANDUVA-FORMOSA-CARRAO',
    'Butantã': 'BUTANTA',
    'Campo Limpo': 'CAMPO LIMPO',
    'Capela do Socorro': 'CAPELA DO SOCORRO',
    'Casa Verde/Cachoeirinha': 'CASA VERDE-CACHOEIRINHA',
    'Cidade Ademar': 'CIDADE ADEMAR',
    'Cidade Tiradentes': 'CIDADE TIRADENTES',
    'Ermelino Matarazzo': 'ERMELINO MATARAZZO',
    'Freguesia/Brasilândia': 'FREGUESIA-BRASILANDIA',
    'Guaianases': 'GUAIANASES',
    'Ipiranga': 'IPIRANGA',
    'Itaim Paulista': 'ITAIM PAULISTA',
    'Itaquera': 'ITAQUERA',
    'Jabaquara': 'JABAQUARA',
    'Jaçanã/Tremembé': 'JACANA-TREMEMBE',
    'Lapa': 'LAPA',
    "M'Boi Mirim": 'M BOI MIRIM',
    'Moóca': 'MOOCA',
    'Parelheiros': 'PARELHEIROS',
    'Penha': 'PENHA',
    'Perus': 'PERUS',
    'Pinheiros': 'PINHEIROS',
    'Pirituba': 'PIRITUBA-JARAGUA',
    'Santana/Tucuruvi': 'SANTANA-TUCURUVI',
    'Santo Amaro': 'SANTO AMARO',
    'Sapopemba': 'SAPOPEMBA',
    'São Mateus': 'SAO MATEUS',
    'São Miguel': 'SAO MIGUEL',
    'Sé': 'SE',
    'Vila Maria/Vila Guilherme': 'VILA MARIA-VILA GUILHERME',
    'Vila Mariana': 'VILA MARIANA',
    'Vila Prudente': 'VILA PRUDENTE'
}

In [67]:
df_mortalidade_ajustado.loc[:, 'SUBPREFEITURA_QLIK'] = (
    df_mortalidade_ajustado.loc[:, 'Subprefeitura residência'].map(subpref_map)
)

df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK
0,Aricanduva/Formosa/Carrão,Doencas infec intest,4,ARICANDUVA-FORMOSA-CARRAO
1,Butantã,Doencas infec intest,5,BUTANTA
2,Campo Limpo,Doencas infec intest,7,CAMPO LIMPO
3,Capela do Socorro,Doencas infec intest,7,CAPELA DO SOCORRO
4,Casa Verde/Cachoeirinha,Doencas infec intest,4,CASA VERDE-CACHOEIRINHA
...,...,...,...,...
2139,Sapopemba,Demais causas de morte,279,SAPOPEMBA
2140,Sé,Demais causas de morte,285,SE
2141,Vila Maria/Vila Guilherme,Demais causas de morte,279,VILA MARIA-VILA GUILHERME
2142,Vila Mariana,Demais causas de morte,185,VILA MARIANA


Para classificar as doenças em evitáveis e/ou tratáveis, foi elabora uma planilha complementar com os pesos de cada categoria para as causas específicas disponíveis no tabnet.

Vamos carregar essa planilha e utilizar como fator de ponderação.

In [68]:
data_path = path.join('dados', 'saude')
mortalidade_filename = 'DOENÇAS EVITAVEIS E-OU TRATAVEIS.xlsx'
mortalidade_file_path = path.join(data_path, mortalidade_filename)

df_evitaveis_trataveis = pd.read_excel(mortalidade_file_path, sheet_name=1)

df_evitaveis_trataveis

,Categorias específicas,Evitáveis,Tratáveis
0,Acid trânsito e transporte terrestres,1.0,NaN
1,Aids,1.0,NaN
2,Alcoolismo,1.0,NaN
3,Anemias,1.0,NaN
4,Aneurisma e dissecção aorta,0.5,0.5
...,...,...,...
62,Quedas acidentais,1.0,NaN
63,Septicemia,NaN,1.0
64,Suicídios,0.5,0.0
65,Tuberculose,0.5,0.5


In [69]:
df_evitaveis_trataveis = df_evitaveis_trataveis.fillna(0)
df_evitaveis_trataveis

,Categorias específicas,Evitáveis,Tratáveis
0,Acid trânsito e transporte terrestres,1.0,0.0
1,Aids,1.0,0.0
2,Alcoolismo,1.0,0.0
3,Anemias,1.0,0.0
4,Aneurisma e dissecção aorta,0.5,0.5
...,...,...,...
62,Quedas acidentais,1.0,0.0
63,Septicemia,0.0,1.0
64,Suicídios,0.5,0.0
65,Tuberculose,0.5,0.5


In [70]:
df_mortalidade_ajustado = df_mortalidade_ajustado.merge(df_evitaveis_trataveis,
                              left_on='Causa específica',
                              right_on='Categorias específicas',
                              how='left')

df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK,Categorias específicas,Evitáveis,Tratáveis
0,Aricanduva/Formosa/Carrão,Doencas infec intest,4,ARICANDUVA-FORMOSA-CARRAO,Doencas infec intest,1.0,0.0
1,Butantã,Doencas infec intest,5,BUTANTA,Doencas infec intest,1.0,0.0
2,Campo Limpo,Doencas infec intest,7,CAMPO LIMPO,Doencas infec intest,1.0,0.0
3,Capela do Socorro,Doencas infec intest,7,CAPELA DO SOCORRO,Doencas infec intest,1.0,0.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,4,CASA VERDE-CACHOEIRINHA,Doencas infec intest,1.0,0.0
...,...,...,...,...,...,...,...
2139,Sapopemba,Demais causas de morte,279,SAPOPEMBA,Demais causas de morte,0.0,0.0
2140,Sé,Demais causas de morte,285,SE,Demais causas de morte,0.0,0.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,279,VILA MARIA-VILA GUILHERME,Demais causas de morte,0.0,0.0
2142,Vila Mariana,Demais causas de morte,185,VILA MARIANA,Demais causas de morte,0.0,0.0


Nessa versão, a planilha não classica a causa específica `Infec menigocóccica`. Por ora, vou atribuir o peso 0 nas duas categorias, mas solicitarei a revisão dessa causa específica aos especialistas do GT Saúde.

In [71]:
df_mortalidade_ajustado.loc[:, 'Evitáveis'] = df_mortalidade_ajustado.loc[:, 'Evitáveis'].fillna(0)
df_mortalidade_ajustado.loc[:, 'Tratáveis'] = df_mortalidade_ajustado.loc[:, 'Tratáveis'].fillna(0)
df_mortalidade_ajustado = df_mortalidade_ajustado\
                            .drop(columns='Categorias específicas')

df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK,Evitáveis,Tratáveis
0,Aricanduva/Formosa/Carrão,Doencas infec intest,4,ARICANDUVA-FORMOSA-CARRAO,1.0,0.0
1,Butantã,Doencas infec intest,5,BUTANTA,1.0,0.0
2,Campo Limpo,Doencas infec intest,7,CAMPO LIMPO,1.0,0.0
3,Capela do Socorro,Doencas infec intest,7,CAPELA DO SOCORRO,1.0,0.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,4,CASA VERDE-CACHOEIRINHA,1.0,0.0
...,...,...,...,...,...,...
2139,Sapopemba,Demais causas de morte,279,SAPOPEMBA,0.0,0.0
2140,Sé,Demais causas de morte,285,SE,0.0,0.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,279,VILA MARIA-VILA GUILHERME,0.0,0.0
2142,Vila Mariana,Demais causas de morte,185,VILA MARIANA,0.0,0.0


In [72]:
df_mortalidade_ajustado = df_mortalidade_ajustado.rename(
    columns={'Evitáveis': 'Qtd evitáveis', 'Tratáveis': 'Qtd tratáveis'}
)

df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK,Qtd evitáveis,Qtd tratáveis
0,Aricanduva/Formosa/Carrão,Doencas infec intest,4,ARICANDUVA-FORMOSA-CARRAO,1.0,0.0
1,Butantã,Doencas infec intest,5,BUTANTA,1.0,0.0
2,Campo Limpo,Doencas infec intest,7,CAMPO LIMPO,1.0,0.0
3,Capela do Socorro,Doencas infec intest,7,CAPELA DO SOCORRO,1.0,0.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,4,CASA VERDE-CACHOEIRINHA,1.0,0.0
...,...,...,...,...,...,...
2139,Sapopemba,Demais causas de morte,279,SAPOPEMBA,0.0,0.0
2140,Sé,Demais causas de morte,285,SE,0.0,0.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,279,VILA MARIA-VILA GUILHERME,0.0,0.0
2142,Vila Mariana,Demais causas de morte,185,VILA MARIANA,0.0,0.0


In [73]:
df_mortalidade_ajustado.loc[:, 'Qtd evitáveis'] = (
    df_mortalidade_ajustado.loc[:, 'Qtd evitáveis']*df_mortalidade_ajustado.loc[:, 'Qtd óbitos']
)

df_mortalidade_ajustado.loc[:, 'Qtd tratáveis'] = (
    df_mortalidade_ajustado.loc[:, 'Qtd tratáveis']*df_mortalidade_ajustado.loc[:, 'Qtd óbitos']
)

/tmp/ipykernel_5635/2988763396.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[4.0 5.0 7.0 ... 0.0 0.0 0.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mortalidade_ajustado.loc[:, 'Qtd evitáveis'] = (
/tmp/ipykernel_5635/2988763396.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.0 0.0 0.0 ... 0.0 0.0 0.0]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_mortalidade_ajustado.loc[:, 'Qtd tratáveis'] = (


In [74]:
df_mortalidade_ajustado

,Subprefeitura residência,Causa específica,Qtd óbitos,SUBPREFEITURA_QLIK,Qtd evitáveis,Qtd tratáveis
0,Aricanduva/Formosa/Carrão,Doencas infec intest,4,ARICANDUVA-FORMOSA-CARRAO,4.0,0.0
1,Butantã,Doencas infec intest,5,BUTANTA,5.0,0.0
2,Campo Limpo,Doencas infec intest,7,CAMPO LIMPO,7.0,0.0
3,Capela do Socorro,Doencas infec intest,7,CAPELA DO SOCORRO,7.0,0.0
4,Casa Verde/Cachoeirinha,Doencas infec intest,4,CASA VERDE-CACHOEIRINHA,4.0,0.0
...,...,...,...,...,...,...
2139,Sapopemba,Demais causas de morte,279,SAPOPEMBA,0.0,0.0
2140,Sé,Demais causas de morte,285,SE,0.0,0.0
2141,Vila Maria/Vila Guilherme,Demais causas de morte,279,VILA MARIA-VILA GUILHERME,0.0,0.0
2142,Vila Mariana,Demais causas de morte,185,VILA MARIANA,0.0,0.0


In [75]:
df_mortalidade_final = (
    df_mortalidade_ajustado
    .drop(columns=['Causa específica', 'Qtd óbitos'])
    .groupby(['Subprefeitura residência', 'SUBPREFEITURA_QLIK'])
    .sum()
    .reset_index()
)

df_mortalidade_final

,Subprefeitura residência,SUBPREFEITURA_QLIK,Qtd evitáveis,Qtd tratáveis
0,Aricanduva/Formosa/Carrão,ARICANDUVA-FORMOSA-CARRAO,385.5,463.5
1,Butantã,BUTANTA,491.0,503.5
2,Campo Limpo,CAMPO LIMPO,789.5,831.5
3,Capela do Socorro,CAPELA DO SOCORRO,859.5,934.0
4,Casa Verde/Cachoeirinha,CASA VERDE-CACHOEIRINHA,471.0,575.0
5,Cidade Ademar,CIDADE ADEMAR,542.0,663.5
6,Cidade Tiradentes,CIDADE TIRADENTES,313.0,357.0
7,Ermelino Matarazzo,ERMELINO MATARAZZO,290.0,363.0
8,Freguesia/Brasilândia,FREGUESIA-BRASILANDIA,639.0,747.0
9,Guaianases,GUAIANASES,393.0,493.5


In [76]:
df_mortalidade_final['Qtd evitáveis'] = (
    df_mortalidade_final['Qtd evitáveis']
    .astype(int)
)
df_mortalidade_final['Qtd tratáveis'] = (
    df_mortalidade_final['Qtd tratáveis']
    .astype(int)
)

df_mortalidade_final

,Subprefeitura residência,SUBPREFEITURA_QLIK,Qtd evitáveis,Qtd tratáveis
0,Aricanduva/Formosa/Carrão,ARICANDUVA-FORMOSA-CARRAO,385,463
1,Butantã,BUTANTA,491,503
2,Campo Limpo,CAMPO LIMPO,789,831
3,Capela do Socorro,CAPELA DO SOCORRO,859,934
4,Casa Verde/Cachoeirinha,CASA VERDE-CACHOEIRINHA,471,575
5,Cidade Ademar,CIDADE ADEMAR,542,663
6,Cidade Tiradentes,CIDADE TIRADENTES,313,357
7,Ermelino Matarazzo,ERMELINO MATARAZZO,290,363
8,Freguesia/Brasilândia,FREGUESIA-BRASILANDIA,639,747
9,Guaianases,GUAIANASES,393,493


In [77]:
df_mortalidade_final = (
    df_mortalidade_final
    .melt(
        id_vars=['Subprefeitura residência', 'SUBPREFEITURA_QLIK'],
        var_name='Evitáveis ou tratáveis',
        value_vars=['Qtd evitáveis', 'Qtd tratáveis'],
        value_name='Qtd óbitos'
    )
)

df_mortalidade_final

,Subprefeitura residência,SUBPREFEITURA_QLIK,Evitáveis ou tratáveis,Qtd óbitos
0,Aricanduva/Formosa/Carrão,ARICANDUVA-FORMOSA-CARRAO,Qtd evitáveis,385
1,Butantã,BUTANTA,Qtd evitáveis,491
2,Campo Limpo,CAMPO LIMPO,Qtd evitáveis,789
3,Capela do Socorro,CAPELA DO SOCORRO,Qtd evitáveis,859
4,Casa Verde/Cachoeirinha,CASA VERDE-CACHOEIRINHA,Qtd evitáveis,471
...,...,...,...,...
59,São Miguel,SAO MIGUEL,Qtd tratáveis,724
60,Sé,SE,Qtd tratáveis,532
61,Vila Maria/Vila Guilherme,VILA MARIA-VILA GUILHERME,Qtd tratáveis,503
62,Vila Mariana,VILA MARIANA,Qtd tratáveis,304


In [78]:
df_mortalidade_final.loc[:, 'Evitáveis ou tratáveis'] = (
    df_mortalidade_final
    .loc[:, 'Evitáveis ou tratáveis']
    .str.removeprefix('Qtd ')
    .str.capitalize()
)

df_mortalidade_final

,Subprefeitura residência,SUBPREFEITURA_QLIK,Evitáveis ou tratáveis,Qtd óbitos
0,Aricanduva/Formosa/Carrão,ARICANDUVA-FORMOSA-CARRAO,Evitáveis,385
1,Butantã,BUTANTA,Evitáveis,491
2,Campo Limpo,CAMPO LIMPO,Evitáveis,789
3,Capela do Socorro,CAPELA DO SOCORRO,Evitáveis,859
4,Casa Verde/Cachoeirinha,CASA VERDE-CACHOEIRINHA,Evitáveis,471
...,...,...,...,...
59,São Miguel,SAO MIGUEL,Tratáveis,724
60,Sé,SE,Tratáveis,532
61,Vila Maria/Vila Guilherme,VILA MARIA-VILA GUILHERME,Tratáveis,503
62,Vila Mariana,VILA MARIANA,Tratáveis,304


## Subprefeitura x Nível de atenção

Para que seja possível relacionar todos os indicadores sem nenhuma conversão nas medidas, precisamos de uma tabela com as dimensões em comum entre todas as tabelas para que seja feita a associação no Qlik. Por isso, vamos criar um dataframe que consistirá no produto cartesiano entre subprefeituras e nível de atenção padronizados.

In [79]:
df_niveis = pd.DataFrame([NIVEL_AB_VIGILANCIA, NIVEL_ESP, NIVEL_OUTROS],
                         columns=[NIVEL_COL_NAME])

df_subp_nivel = (
    df_mortalidade_final[['SUBPREFEITURA_QLIK']]
    .drop_duplicates()
    .merge(df_niveis, how='cross')
)

df_subp_nivel

,SUBPREFEITURA_QLIK,Nível de atenção padronizado
0,ARICANDUVA-FORMOSA-CARRAO,Atenção Básica e Vigilância
1,ARICANDUVA-FORMOSA-CARRAO,Atenção Especializada e Suporte profilático te...
2,ARICANDUVA-FORMOSA-CARRAO,Outros níveis de atenção
3,BUTANTA,Atenção Básica e Vigilância
4,BUTANTA,Atenção Especializada e Suporte profilático te...
...,...,...
91,VILA MARIANA,Atenção Especializada e Suporte profilático te...
92,VILA MARIANA,Outros níveis de atenção
93,VILA PRUDENTE,Atenção Básica e Vigilância
94,VILA PRUDENTE,Atenção Especializada e Suporte profilático te...


In [80]:
df_subp_nivel['subprefeitura_nivel'] = (
    df_subp_nivel['SUBPREFEITURA_QLIK'] +
    ' | ' +
    df_subp_nivel['Nível de atenção padronizado']
)

df_subp_nivel

,SUBPREFEITURA_QLIK,Nível de atenção padronizado,subprefeitura_nivel
0,ARICANDUVA-FORMOSA-CARRAO,Atenção Básica e Vigilância,ARICANDUVA-FORMOSA-CARRAO | Atenção Básica e V...
1,ARICANDUVA-FORMOSA-CARRAO,Atenção Especializada e Suporte profilático te...,ARICANDUVA-FORMOSA-CARRAO | Atenção Especializ...
2,ARICANDUVA-FORMOSA-CARRAO,Outros níveis de atenção,ARICANDUVA-FORMOSA-CARRAO | Outros níveis de a...
3,BUTANTA,Atenção Básica e Vigilância,BUTANTA | Atenção Básica e Vigilância
4,BUTANTA,Atenção Especializada e Suporte profilático te...,BUTANTA | Atenção Especializada e Suporte prof...
...,...,...,...
91,VILA MARIANA,Atenção Especializada e Suporte profilático te...,VILA MARIANA | Atenção Especializada e Suporte...
92,VILA MARIANA,Outros níveis de atenção,VILA MARIANA | Outros níveis de atenção
93,VILA PRUDENTE,Atenção Básica e Vigilância,VILA PRUDENTE | Atenção Básica e Vigilância
94,VILA PRUDENTE,Atenção Especializada e Suporte profilático te...,VILA PRUDENTE | Atenção Especializada e Suport...


# Armazenamento dos dados

Finalmente, salvamos os arquivos como csv para utilizarmos no Qlik Sense.

In [81]:
base_path = path.join('data_output', 'saude')

if not path.exists(base_path):
    makedirs(base_path)

for name, df in [('consultas', df_consultas_final),
                 ('leitos-internacao-complementares', df_leitos_final),
                 ('despesas', df_despesas_final),
                 ('mortalidade', df_mortalidade_final),
                 ('subprefeitura-nivel', df_subp_nivel)]:
    
    filepath = path.join(base_path, f'{name}.csv')

    df.to_csv(filepath,
              index=False,
              sep=';',
              decimal=',')